In [4]:
#import all the necessary libraries
import networkx as nx
import numpy as np
import pandas as pd
import networkx.algorithms.community as nx_comm
import math
import matplotlib.pyplot as plt
import random
print("Libraries successfully imported")
#all the side functions
def generate_chrom(nodes_length,Adj):
    chrom = np.array([],dtype=int)
    for x in range(nodes_length):
        rand = np.random.randint(0,nodes_length)
        while Adj[x,rand] != 1:
            rand = np.random.randint(0,nodes_length)
        chrom = np.append(chrom,rand)
    return chrom

def merge_subsets(sub):
    arr =[]
    to_skip=[]
    for s in range(len(sub)):
        if sub[s] not in to_skip:
            new = sub[s]
            for x in sub:
                if sub[s] & x:
                    new = new | x
                    to_skip.append(x)
            arr.append(new)
    return arr

def find_subsets(chrom):
    sub = [{x,chrom[x]} for x in range(len(chrom))]
    result=sub
    i=0
    while i<2*len(result):
        candidate = merge_subsets(result)
        if candidate != result:
            result = candidate
        else:
            break
        result=candidate
        i+=1
    return result

def community_score(chrom,subsets,r,Adj):
    matrix = Adj.toarray()
    CS=0
    for s in subsets:
        submatrix = np.zeros((len(chrom),len(chrom)),dtype=int)
        for i in s:
            for j in s:
                submatrix[i][j]=matrix[i][j]
        M=0
        v=0
        PS=0
        for row in list(s):
            ki = np.sum(matrix[row])
            kiin = np.sum(submatrix[row])
            kiout = ki - kiin
            P= kiin/ki
            PS+=P
            row_mean = kiin/len(s)
            v+=np.sum(submatrix[row])
            M+=(row_mean**r)/len(s)
        CS+=M*v
    OS= 0.5*CS/len(subsets) + 0.5*(1/PS)*len(subsets)  #Overall score is calculated by maximizing CS and min PS
    return OS
def score_calc(chrom,subsets,r,Adj):
    matrix = Adj.toarray()
    CS=0
    for s in subsets:
        submatrix = np.zeros((len(chrom),len(chrom)),dtype=int)
        for i in s:
            for j in s:
                submatrix[i][j]=matrix[i][j]
        M=0
        v=0
        PS=0
        for row in list(s):
            ki = np.sum(matrix[row])
            kiin = np.sum(submatrix[row])
            kiout = ki - kiin
            P= kiin/ki
            PS+=P
            row_mean = kiin/len(s)
            v+=np.sum(submatrix[row])
            M+=(row_mean**r)/len(s)
        CS+=M*v
#     OS= 0.5*CS/len(subsets) + 0.5*(1/PS)*len(subsets)  #Overall score is calculated by maximizing CS and min PS
    return CS/len(subsets), PS/len(subsets)


def gwolagao(xalpha, xbeta, xgamma, wolf, Adj,a):
    A1 = (2*a*np.random.rand()) - a
    C1 = 2*np.random.rand()
    Dalpha = abs(C1*xalpha - wolf['chrom'])
    X1 = abs(xalpha - (A1*Dalpha))
    A2 = (2*a*np.random.rand()) - a
    C2 = 2*np.random.rand()
    Dbeta = abs(C2*xbeta - wolf['chrom'])
    X2 = abs(xbeta - (A2*Dbeta))
    A3 = (2*a*np.random.rand()) - a
    C3 = 2*np.random.rand()
    Dgamma = abs(C3*xgamma - wolf['chrom'])
    X3 = abs(xgamma - (A3*Dgamma))
    Xnew = (X1+X2+X3)/3
    Xnew = np.array([int(i) for i in Xnew])
    for i in range(0,len(Xnew)):
        if Xnew[i] < 0 or Xnew[i] >= Adj.shape[0] or Adj[i,Xnew[i]]==0:
            Xnew[i] = xalpha[i]
    wolf["chrom"] = Xnew
    wolf["subsets"]= find_subsets(Xnew)
    wolf["community_score"] = community_score(wolf["chrom"],wolf["subsets"], 1.5, Adj)
    return wolf



def roulette_selection(df_elites):
    prob = np.random.random_sample()
    sum_cs=np.sum(df_elites["community_score"])
    x=0
    selected = 0
    for i in df_elites.index:
        x += df_elites["community_score"][i]
        X=x/sum_cs
        if prob < X:
            chosen=i
            break
    return chosen

def uniform_crossover_GE(parent_1,parent_2,crossover_rate):
    if np.random.random_sample() < crossover_rate:
        length = len(parent_1)
        mask = np.random.randint(2, size=length)
        child = np.zeros(length,dtype=int)
        for i in range(len(mask)):
            if mask[i] == 1:
                child[i]=parent_1[i]
            else:
                child[i]=parent_2[i]
        return child
    else:
        return np.array([])

def crossover_DE(parent,trial,crossover_rate):
    if np.random.random_sample() >= crossover_rate:
        length = len(parent['chrom'])
        mask = np.random.randint(2, size=length)
        child = np.zeros(length,dtype=int)
        for i in range(len(mask)):
            if mask[i] == 1:
                child[i]=trial[i]
            else:
                child[i]=parent['chrom'][i]
        return child
    else:
        return trial
    
def mutation_GE(chrom,Adj,mutation_rate):
    if np.random.random_sample() < mutation_rate:
        chrom = chrom
        neighbor = []
        while len(neighbor) < 2:
            mutant = np.random.randint(1,len(chrom))
            row = Adj[mutant].toarray()[0]
            neighbor = [i for i in range(len(row)) if row[i]==1]
            if len(neighbor) > 1:
                neighbor.remove(chrom[mutant])
                to_change=int(np.floor(np.random.random_sample()*(len(neighbor))))
                chrom[mutant]=neighbor[to_change]
                neighbor.append(chrom[mutant])
    return chrom

def mutation_DE(parent , target , rand1 , rand2 ,Adj,mutation_rate):
    trial = []
    for i in range(0,len(parent['chrom'])):
        temp = target['chrom'][i] + mutation_rate*(rand1['chrom'][i] - rand2['chrom'][i])
        temp = int(temp)
        if temp < 0 or temp >= Adj.shape[0] or Adj[i,temp]==0:
            temp = target['chrom'][i]
        trial.append(temp)
    return trial

#NMI
def entropy(nums):
    z = np.bincount(nums)
    N = len(nums)
    assert nums.shape == (N, )
    ent = 0.0
    for e in z:
        if e != 0:
            p = float(e) / N
            ent += p*math.log(p)
    assert ent <= 0
    ent = -ent
    assert ent >=0
    return ent

def computeNMI(clusters, classes):
    assert clusters.shape == classes.shape
    A = np.c_[(clusters, classes)]
    A = np.array(A)
    N = A.shape[0]
    assert A.shape == (N, 2)
    H_clusters = entropy(A[:, 0])
    H_classes = entropy(A[:, 1])
    NMI = 0.0
    for k in np.unique(A[:, 0]):
        z = A[A[:, 0] == k, 1]
        len_wk = len(z)
        t = A[:, 1]
        for e in np.unique(z):
            wk_cj=len(z[z==e])
            len_cj=len(t[t == e])
            assert wk_cj <= len_cj
            numerator= (float(wk_cj) / float(N)) * math.log( (N*wk_cj) / float(len_wk * len_cj)  )
            NMI += numerator
    NMI /= float((H_clusters + H_classes) * 0.5)

    assert (NMI > 0.0 or abs(NMI) < 1e-10) and (NMI < 1.0 or abs(NMI - 1.0) < 1e-10)
    return NMI

def check_res(res,Adj):
    print(res)
    for i in range(0,len(res)):
        if Adj[i,res[i]] ==0:
            return False
    return True

def print_graph(graph, result, nodes):
    colors = []
    for i in range(0, len(result)):
        colors.append('#%06X' % np.random.randint(0, 0xFFFFFF))
    col_arr = ['']*len(nodes)
    for i in range(0,len(result)):
        for j in result[i]:
            col_arr[j-1] = colors[i]
    nx.draw(graph, with_labels=True,node_color = col_arr)
    plt.show()
    
def ShrinkingEncircling(i,P_se,Adj):
    
    N = Adj.shape[0]
    n = N*P_se
    color = np.zeros(N,dtype=int)
    val = 0
    for j in i["subsets"]:
        for k in j:
            color[k] = val
        val+=1
    
    
        
    for z in range(0,int(n)):
        index = random.randrange(0,N)
        freq = np.zeros(val, dtype = int)
        for k in range(0, N):
            if Adj[index,k] == 1:
                freq[color[k]]+=1
        max_freq = np.argmax(freq)
        
        for k in range(0, len(color)):
            if(color[k] == max_freq and Adj[index,k] == 1):
                i["chrom"][index] = k
                break
    i["subsets"] = find_subsets(i["chrom"])
    i["community_score"] = community_score(i["chrom"],i["subsets"],1.5,Adj)
    return i

def RandomSearchingOperation(i,P_rs,Adj):
    neigh = []
#     color = AssignColor(i,Adj)
    for j in i["chrom"]:
        r1 = np.random.rand()
        if r1<=P_rs:
            for k in range(0,Adj.shape[0]-1):
                if Adj[j,k]==1:
                    if j!=k:
                        neigh.append(k)
            if len(neigh)!=0:
                ind = random.randrange(0,len(neigh))
                i["chrom"][j] = neigh[ind]
                neigh.clear()
    i["subsets"] = find_subsets(i["chrom"])
    i["community_score"] = community_score(i["chrom"],i["subsets"],1.5,Adj)
    return i   
           

    
def SpiralUpdatingOperation(i,P_su,gbest,Adj):
    N = len(gbest["chrom"])
    n = N*P_su
    for k in range(0,int(n)):
        r = random.randrange(0,N)
        i["chrom"][r] = gbest["chrom"][r]
    
    i["subsets"] = find_subsets(i["chrom"])
    i["community_score"] = community_score(i["chrom"],i["subsets"],1.5,Adj)
    return i
    
def ChangingVelocity(particle, globalbest, localbest, Adj, W, c1, c2,r):
    CC = particle['chrom']
    LBC = localbest['chrom']
    GBC = globalbest['chrom']
                    
#                 velocity
    newvel=0
    for gene in range(0,len(particle['chrom'])):
        vel = (W*particle['vel']) + c1*(np.random.rand())*(LBC[gene]-CC[gene]) + c2*(np.random.rand())*(GBC[gene]-CC[gene])
        newgene = CC[gene] + int(vel)
        if newgene>=0 and newgene<Adj.shape[0] and Adj[gene,newgene] == 1:
            CC[gene] = newgene
        else:
            vel = particle['vel']
        newvel+=vel
    newvel /= len(particle['chrom'])
    particle['vel'] = newvel
    particle['chrom'] = CC
    particle['subsets'] = find_subsets(CC)
    particle['community_score'] = community_score(CC,particle['subsets'],r,Adj)
    return particle

print("All functions are ready to use")
nodes = []
edges = np.loadtxt('C-elegans-frontal.txt')
for i in edges:
    for j in i:
        if j not in nodes:
            nodes.append(int(j))
print("Data ready")
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

Libraries successfully imported
All functions are ready to use
Data ready


In [5]:
# PSO
def PSO(nodes,edges,population=20,generation=100,r=1.5):
    graph=nx.Graph() 
    graph.add_nodes_from(nodes) #adds nodes
    graph.add_edges_from(edges) #add edges
    Adj = nx.adjacency_matrix(graph) 
    nodes_length = len(graph.nodes())
#     nx.draw(graph, with_labels=True,node_color = "red")
#     plt.show()
    d = {"chrom":[generate_chrom(nodes_length,Adj) for n in range(population)]}
    dframe = pd.DataFrame(data= d)
    dframe["subsets"] = dframe["chrom"].apply(find_subsets)
    dframe["community_score"]=dframe.apply(lambda x: community_score(x["chrom"],x["subsets"],r,Adj),axis=1)
    W = 0.8
    c1 = 0.3
    c2 = 0.7
    V = [np.random.randint(-nodes_length/3, nodes_length/3)]*len(dframe)
    dframe["vel"] = V
    population_count = population
    gen = 0
    
    
    
    globalbest=[]
    
    
    
    
    
    
    localbest= []
    while gen < generation:
        for i in range(int(np.floor(population/10))):
#         for i in range(population):
        
            localbest = dframe
            if gen!=0:
                for index , particle in dframe.iterrows():
                    
                    
                    
                    
                    
                    
                    
                    
                    
                    
                    
#                 ///////////////////////////
                    CC = particle['chrom']
                    LBC = localbest.loc[index]['chrom']
                    GBC = globalbest['chrom']
                    
#                 velocity
                    newvel=0
                    for gene in range(0,len(particle['chrom'])):
                        vel = (W*particle['vel']) + c1*(np.random.rand())*(LBC[gene]-CC[gene]) + c2*(np.random.rand())*(GBC[gene]-CC[gene])
                        newgene = CC[gene] + int(vel)
                        if newgene>=0 and newgene<Adj.shape[0] and Adj[gene,newgene] == 1:
                            CC[gene] = newgene
                        else:
                            vel = particle['vel']
                        newvel+=vel
                    newvel /= len(particle['chrom'])
                    newsub = find_subsets(CC)
                    newscore = community_score(CC,newsub,r,Adj)
#                     ..............
                    if(newscore>localbest.loc[index]['community_score']):
                        dframe.at[index,'subsets'] = newsub
                        dframe.at[index,'chrom'] = [g for g in CC]
                        dframe.at[index,'community_score'] = newscore
                    dframe.at[index,'vel'] = newvel
#             ...................................
            
    
    
    
    
    
    
    
    
    
    
    
    
    
            globalbest = dframe.sort_values("community_score",ascending=False).index[0]
            globalbest = dframe.loc[globalbest]                   
        gen +=1   
    
    
    
    
    
    
    
    
    
    sorted_df = dframe.sort_values("community_score",ascending=False).index[0]
    res = dframe.loc[sorted_df]
    nodes_subsets = res["subsets"]
    res = res['chrom']
#     istrue = check_res(res,Adj)
#     print(istrue)
    CS, PS = score_calc(res, nodes_subsets, r, Adj)
    nodes_list = list(graph.nodes())
    result = []
    for subs in nodes_subsets:
        subset = []
        for n in subs:
            subset.append(nodes_list[n])
        result.append(subset)
    NMI = 0
    clu = dframe.loc[sorted_df]
    clu = clu['chrom']
    clu = np.array(clu)
    for index, target in dframe.iterrows():
        temp = np.array(target['chrom'])
        x = computeNMI(clu,temp)
        NMI += x
    NMI /= len(dframe)
    modularity = nx_comm.modularity(graph, result)
    return result, NMI, modularity, CS, PS
print("PSO loaded to use")

PSO loaded to use


In [6]:
NMI_arr_PSO = []
mod_arr_PSO = []
bst_arr_PSO = []
C_S_arr_PSO = []
P_S_arr_PSO = []

for i in range(0,20):
    print("PSO",i+1, end=" ")
    best_community, NMI, modularity,CS, PS = PSO(nodes,edges)
    NMI_arr_PSO.append(NMI)
    mod_arr_PSO.append(modularity)
    bst_arr_PSO.append(best_community)
    C_S_arr_PSO.append(CS)
    P_S_arr_PSO.append(PS)
    

NMI_avg_PSO = sum(NMI_arr_PSO)/len(NMI_arr_PSO)
mod_avg_PSO = sum(mod_arr_PSO)/len(mod_arr_PSO)
C_S_avg_PSO = sum(C_S_arr_PSO)/len(C_S_arr_PSO)
P_S_avg_PSO = sum(P_S_arr_PSO)/len(P_S_arr_PSO)


NMI_bst_PSO = max(NMI_arr_PSO)  #best
mod_bst_PSO = max(mod_arr_PSO)
C_S_bst_PSO = max(C_S_arr_PSO)
P_S_bst_PSO = max(P_S_arr_PSO)

NMI_lst_PSO = min(NMI_arr_PSO)  #least
mod_lst_PSO = min(mod_arr_PSO)
C_S_lst_PSO = min(C_S_arr_PSO)
P_S_lst_PSO = min(P_S_arr_PSO)




print("PSO")
print('NMI avg: ', NMI_avg_PSO)
print('NMI max: ', NMI_bst_PSO)
print('NMI min: ', NMI_lst_PSO)
print()
print('mod avg: ', mod_avg_PSO)
print('mod max: ', mod_bst_PSO)
print('mod min: ', mod_lst_PSO)
print()
print('C_S avg: ', C_S_avg_PSO)
print('C_S max: ', C_S_bst_PSO)
print('C_S min: ', C_S_lst_PSO)
print()
print('P_S avg: ', P_S_avg_PSO)
print('P_S max: ', P_S_bst_PSO)
print('P_S min: ', P_S_lst_PSO)
print("Done")

PSO 1 PSO 2 PSO 3 PSO 4 PSO 5 PSO 6 PSO 7 PSO 8 PSO 9 PSO 10 PSO 11 PSO 12 PSO 13 PSO 14 PSO 15 PSO 16 PSO 17 PSO 18 PSO 19 PSO 20 PSO
NMI avg:  0.8769523885806013
NMI max:  0.9126998792154492
NMI min:  0.8525443783812381

mod avg:  0.1614865065290305
mod max:  0.21829611690598322
mod min:  0.035148494922336036

C_S avg:  4.0314361337293745
C_S max:  5.036069067684003
C_S min:  3.0121425084519893

P_S avg:  0.014579591391237923
P_S max:  0.022727272727272728
P_S min:  0.009572649572649573
Done


In [7]:
def WOA(nodes,edges,population=20,generation=100,P_se=0.05, P_su = 0.05, P_rs = 0.1):
    graph=nx.Graph() 
    graph.add_nodes_from(nodes) #adds nodes
    graph.add_edges_from(edges) #add edges
    Adj = nx.adjacency_matrix(graph) 
    nodes_length = len(graph.nodes())
#     nx.draw(graph, with_labels=True,node_color = "red")
#     plt.show()
    d = {"chrom":[generate_chrom(nodes_length,Adj) for n in range(population)]}
    dframe = pd.DataFrame(data= d)
    dframe["subsets"] = dframe["chrom"].apply(find_subsets)
    dframe["community_score"]=dframe.apply(lambda x: community_score(x["chrom"],x["subsets"],1.5,Adj),axis=1)
    population_count = population
    gen = 0
    
    
    
    
    
    globalbest=[]
    globalbest = dframe.sort_values("community_score",ascending=False).index[0]
    globalbest = dframe.loc[globalbest]

    while gen < generation:
       
        for i in range(int(np.floor(population/10))):

            r = np.random.rand()
            a = (2-2*gen)/generation
            M = (2*r-1)*a 
            if r < 0.5:
                if abs(M) < 1:
                    dframe.loc[i] = ShrinkingEncircling(dframe.loc[i],P_se, Adj)
                elif abs(M)>=1:
                    dframe.loc[i] = RandomSearchingOperation(dframe.loc[i],P_rs,Adj)
            elif r>=0.5:
                dframe.loc[i] = SpiralUpdatingOperation(dframe.loc[i],P_su,globalbest,Adj)
        
        

        
        tempbest = dframe.sort_values("community_score",ascending=False).index[0]
        tempbest = dframe.loc[tempbest]
        
        if globalbest["community_score"]<tempbest["community_score"]:
            globalbest = tempbest
                           
        gen +=1   

    res = globalbest
    nodes_subsets = res["subsets"]
#     score = res["community_score"]
    res = res['chrom']
    CS, PS = score_calc(res, nodes_subsets, r, Adj)
    
#     istrue = check_res(res,Adj)
#     print(istrue)
    nodes_list = list(graph.nodes())
    result = []
    for subs in nodes_subsets:
        subset = []
        for n in subs:
            subset.append(nodes_list[n])
        result.append(subset)
    NMI = 0
    clu = globalbest
    clu = clu['chrom']
    clu = np.array(clu)
    for index, target in dframe.iterrows():
        temp = np.array(target['chrom'])
        x = computeNMI(clu,temp)
        NMI += x
    NMI /= len(dframe)
    modularity = nx_comm.modularity(graph, result)
    return result, NMI, modularity, CS,PS
print("WOA loaded to use")

WOA loaded to use


In [8]:
NMI_arr_WOA = []
mod_arr_WOA = []
bst_arr_WOA = []
C_S_arr_WOA = []
P_S_arr_WOA = []

for i in range(0,20):
    print("WOA",i+1, end=" ")
    best_community, NMI, modularity,CS, PS = WOA(nodes,edges)
    NMI_arr_WOA.append(NMI)
    mod_arr_WOA.append(modularity)
    bst_arr_WOA.append(best_community)
    C_S_arr_WOA.append(CS)
    P_S_arr_WOA.append(PS)
    

NMI_avg_WOA = sum(NMI_arr_WOA)/len(NMI_arr_WOA)
mod_avg_WOA = sum(mod_arr_WOA)/len(mod_arr_WOA)
C_S_avg_WOA = sum(C_S_arr_WOA)/len(C_S_arr_WOA)
P_S_avg_WOA = sum(P_S_arr_WOA)/len(P_S_arr_WOA)


NMI_bst_WOA = max(NMI_arr_WOA)  #best
mod_bst_WOA = max(mod_arr_WOA)
C_S_bst_WOA = max(C_S_arr_WOA)
P_S_bst_WOA = max(P_S_arr_WOA)

NMI_lst_WOA = min(NMI_arr_WOA)  #least
mod_lst_WOA = min(mod_arr_WOA)
C_S_lst_WOA = min(C_S_arr_WOA)
P_S_lst_WOA = min(P_S_arr_WOA)




print("WOA")
print('NMI avg: ', NMI_avg_WOA)
print('NMI max: ', NMI_bst_WOA)
print('NMI min: ', NMI_lst_WOA)
print()
print('mod avg: ', mod_avg_WOA)
print('mod max: ', mod_bst_WOA)
print('mod min: ', mod_lst_WOA)
print()
print('C_S avg: ', C_S_avg_WOA)
print('C_S max: ', C_S_bst_WOA)
print('C_S min: ', C_S_lst_WOA)
print()
print('P_S avg: ', P_S_avg_WOA)
print('P_S max: ', P_S_bst_WOA)
print('P_S min: ', P_S_lst_WOA)
print("Done")

WOA 1 WOA 2 WOA 3 WOA 4 WOA 5 WOA 6 WOA 7 WOA 8 WOA 9 WOA 10 WOA 11 WOA 12 WOA 13 WOA 14 WOA 15 WOA 16 WOA 17 WOA 18 WOA 19 WOA 20 WOA
NMI avg:  0.8445024727224777
NMI max:  0.8798297487265477
NMI min:  0.8044433016994976

mod avg:  0.16634651852134358
mod max:  0.21206583483237249
mod min:  0.121907371034962

C_S avg:  26.385824874848318
C_S max:  55.48641330188887
C_S min:  9.528560597657883

P_S avg:  0.016220421171576634
P_S max:  0.022675736961451247
P_S min:  0.011507936507936509
Done


In [9]:
def GWO(nodes,edges,population=15,generation=30,r=1.5):
    graph=nx.Graph() 
    graph.add_nodes_from(nodes) #adds nodes
    graph.add_edges_from(edges) #add edges
    Adj = nx.adjacency_matrix(graph) 
    nodes_length = len(graph.nodes())
#     nx.draw(graph, with_labels=True,node_color = "red")
#     plt.show()
    d = {"chrom":[generate_chrom(nodes_length,Adj) for n in range(population)]}
    dframe = pd.DataFrame(data= d)
    dframe["subsets"] = dframe["chrom"].apply(find_subsets)
    dframe["community_score"]=dframe.apply(lambda x: community_score(x["chrom"],x["subsets"],r,Adj),axis=1)
    gen = 0
    population_count = population
    while gen < generation:
        for i in range(int(np.floor(population/10))):
            elites = dframe.sort_values("community_score",ascending=True)[int(np.floor(population/10)):]
            best3 = dframe.sort_values("community_score", ascending=False)[0:3]
            xalpha = dframe.loc[best3.index[0]]['chrom']
            xbeta = dframe.loc[best3.index[1]]['chrom']
            xgamma = dframe.loc[best3.index[2]]['chrom']
            a = 2*(1-(gen/generation))
            for index , wolf in elites.iterrows():
                A1 = (2*a*np.random.rand()) - a
                C1 = 2*np.random.rand()
                Dalpha = abs(C1*xalpha - wolf['chrom'])
                X1 = abs(xalpha - (A1*Dalpha))
                A2 = (2*a*np.random.rand()) - a
                C2 = 2*np.random.rand()
                Dbeta = abs(C2*xbeta - wolf['chrom'])
                X2 = abs(xbeta - (A2*Dbeta))
                A3 = (2*a*np.random.rand()) - a
                C3 = 2*np.random.rand()
                Dgamma = abs(C3*xgamma - wolf['chrom'])
                X3 = abs(xgamma - (A3*Dgamma))
                Xnew = (X1+X2+X3)/3
                Xnew = np.array([int(i) for i in Xnew])
                for i in range(0,len(Xnew)):
                    if Xnew[i] < 0 or Xnew[i] >= Adj.shape[0] or Adj[i,Xnew[i]]==0:
                        Xnew[i] = xalpha[i]
                subsetnew = find_subsets(Xnew)
                fscorenew = community_score(Xnew,subsetnew, r, Adj)
                dframe.loc[population_count]=[Xnew,subsetnew,fscorenew]
                population_count += 1
        dfsorted = dframe.sort_values("community_score",ascending=False)
        to_drop = dfsorted.index[population:]
        dframe.drop(to_drop,inplace=True)
        gen +=1   
    sorted_df = dframe.sort_values("community_score",ascending=False).index[0]
    res = dframe.loc[sorted_df]
    res = res['chrom']
#     istrue = check_res(res,Adj)
#     print(istrue)
    res_subsets = find_subsets(res)
    nodes_subsets = res_subsets
    CS, PS = score_calc(res, nodes_subsets, r, Adj)
    nodes_list = list(graph.nodes())
    result = []
    for subs in nodes_subsets:
        subset = []
        for n in subs:
            subset.append(nodes_list[n])
        result.append(subset)
    NMI = 0
    clu = dframe.loc[sorted_df]
    clu = clu['chrom']
    clu = np.array(clu)
    for index, target in dframe.iterrows():
        temp = np.array(target['chrom'])
        x = computeNMI(clu,temp)
        NMI += x
    NMI /= len(dframe)
    modularity = nx_comm.modularity(graph, result)
    return result, NMI, modularity, CS, PS
print("GWO loaded to use")

GWO loaded to use


In [10]:
NMI_arr_GWO = []
mod_arr_GWO = []
bst_arr_GWO = []
C_S_arr_GWO = []
P_S_arr_GWO = []

for i in range(0,20):
    print("GWO",i+1, end=" ")
    best_community, NMI, modularity,CS, PS = GWO(nodes,edges)
    NMI_arr_GWO.append(NMI)
    mod_arr_GWO.append(modularity)
    bst_arr_GWO.append(best_community)
    C_S_arr_GWO.append(CS)
    P_S_arr_GWO.append(PS)
    

NMI_avg_GWO = sum(NMI_arr_GWO)/len(NMI_arr_GWO)
mod_avg_GWO = sum(mod_arr_GWO)/len(mod_arr_GWO)
C_S_avg_GWO = sum(C_S_arr_GWO)/len(C_S_arr_GWO)
P_S_avg_GWO = sum(P_S_arr_GWO)/len(P_S_arr_GWO)


NMI_bst_GWO = max(NMI_arr_GWO)  #best
mod_bst_GWO = max(mod_arr_GWO)
C_S_bst_GWO = max(C_S_arr_GWO)
P_S_bst_GWO = max(P_S_arr_GWO)

NMI_lst_GWO = min(NMI_arr_GWO)  #least
mod_lst_GWO = min(mod_arr_GWO)
C_S_lst_GWO = min(C_S_arr_GWO)
P_S_lst_GWO = min(P_S_arr_GWO)




print("GWO")
print('NMI avg: ', NMI_avg_GWO)
print('NMI max: ', NMI_bst_GWO)
print('NMI min: ', NMI_lst_GWO)
print()
print('mod avg: ', mod_avg_GWO)
print('mod max: ', mod_bst_GWO)
print('mod min: ', mod_lst_GWO)
print()
print('C_S avg: ', C_S_avg_GWO)
print('C_S max: ', C_S_bst_GWO)
print('C_S min: ', C_S_lst_GWO)
print()
print('P_S avg: ', P_S_avg_GWO)
print('P_S max: ', P_S_bst_GWO)
print('P_S min: ', P_S_lst_GWO)
print("Done")

GWO 1 GWO 2 GWO 3 GWO 4 GWO 5 GWO 6 GWO 7 GWO 8 GWO 9 GWO 10 GWO 11 GWO 12 GWO 13 GWO 14 GWO 15 GWO 16 GWO 17 GWO 18 GWO 19 GWO 20 GWO
NMI avg:  0.9882551668724531
NMI max:  1.0
NMI min:  0.9684549393957422

mod avg:  0.17038995145867636
mod max:  0.20493295110483956
mod min:  0.1388661119692183

C_S avg:  3.3020768919256467
C_S max:  4.58962595592992
C_S min:  2.5917911202561013

P_S avg:  0.009142790113691414
P_S max:  0.01388888888888889
P_S min:  0.005871212121212121
Done


In [11]:
def GWOWOAlinear(nodes,edges,population=20,generation=100,r=1.5,P_se=0.05, P_su = 0.05, P_rs = 0.1):
    graph=nx.Graph() 
    graph.add_nodes_from(nodes) #adds nodes
    graph.add_edges_from(edges) #add edges
    Adj = nx.adjacency_matrix(graph) 
    nodes_length = len(graph.nodes())
#     nx.draw(graph, with_labels=True,node_color = "red")
#     plt.show()
    d = {"chrom":[generate_chrom(nodes_length,Adj) for n in range(population)]}
    dframe = pd.DataFrame(data= d)
    dframe["subsets"] = dframe["chrom"].apply(find_subsets)
    dframe["community_score"]=dframe.apply(lambda x: community_score(x["chrom"],x["subsets"],r,Adj),axis=1)
    gen = 0
    globalbest=[]
    globalbest = dframe.sort_values("community_score",ascending=False).index[0]
    globalbest = dframe.loc[globalbest]
    population_count = population
    while gen < generation:
        for i in range(int(np.floor(population/10))):
            elites = dframe.sort_values("community_score",ascending=True)[int(np.floor(population/10)):]
            best3 = dframe.sort_values("community_score", ascending=False)[0:3]
            xalpha = dframe.loc[best3.index[0]]['chrom']
            xbeta = dframe.loc[best3.index[1]]['chrom']
            xgamma = dframe.loc[best3.index[2]]['chrom']
            a = 2*(1-(gen/generation))
            for index , wolf in elites.iterrows():
                A1 = (2*a*np.random.rand()) - a
                C1 = 2*np.random.rand()
                Dalpha = abs(C1*xalpha - wolf['chrom'])
                X1 = abs(xalpha - (A1*Dalpha))
                A2 = (2*a*np.random.rand()) - a
                C2 = 2*np.random.rand()
                Dbeta = abs(C2*xbeta - wolf['chrom'])
                X2 = abs(xbeta - (A2*Dbeta))
                A3 = (2*a*np.random.rand()) - a
                C3 = 2*np.random.rand()
                Dgamma = abs(C3*xgamma - wolf['chrom'])
                X3 = abs(xgamma - (A3*Dgamma))
                Xnew = (X1+X2+X3)/3
                Xnew = np.array([int(i) for i in Xnew])
                for i in range(0,len(Xnew)):
                    if Xnew[i] < 0 or Xnew[i] >= Adj.shape[0] or Adj[i,Xnew[i]]==0:
                        Xnew[i] = xalpha[i]
                subsetnew = find_subsets(Xnew)
                fscorenew = community_score(Xnew,subsetnew, r, Adj)
                dframe.loc[population_count]=[Xnew,subsetnew,fscorenew]
                population_count += 1
        for i in dframe.index:
            
            r = np.random.rand()
            a = (2-2*gen)/generation
            M = (2*r-1)*a 
            if r < 0.5:
                if abs(M) < 1:
                    dframe.loc[i] = ShrinkingEncircling(dframe.loc[i],P_se, Adj)
                elif abs(M)>=1:
                    dframe.loc[i] = RandomSearchingOperation(dframe.loc[i],P_rs,Adj)
            elif r>=0.5:
                dframe.loc[i] = SpiralUpdatingOperation(dframe.loc[i],P_su,globalbest,Adj)
        tempbest = dframe.sort_values("community_score",ascending=False).index[0]
        tempbest = dframe.loc[tempbest]
        
        if globalbest["community_score"]<tempbest["community_score"]:
            globalbest = tempbest
        dfsorted = dframe.sort_values("community_score",ascending=False)
        to_drop = dfsorted.index[population:]
        dframe.drop(to_drop,inplace=True)
        gen +=1   
    sorted_df = dframe.sort_values("community_score",ascending=False).index[0]
    res = dframe.loc[sorted_df]
    res = res['chrom']
    res_subsets = find_subsets(res)
    nodes_subsets = res_subsets
    nodes_list = list(graph.nodes())
    CS, PS = score_calc(res, nodes_subsets, r, Adj)
    result = []
    for subs in nodes_subsets:
        subset = []
        for n in subs:
            subset.append(nodes_list[n])
        result.append(subset)
    NMI = 0
    clu = dframe.loc[sorted_df]
    clu = clu['chrom']
    clu = np.array(clu)
    for index, target in dframe.iterrows():
        temp = np.array(target['chrom'])
        x = computeNMI(clu,temp)
        NMI += x
    NMI /= len(dframe)
    modularity = nx_comm.modularity(graph, result)
    return result, NMI, modularity, CS, PS
print("GWOWOAlinear ready to use")

GWOWOAlinear ready to use


In [12]:
NMI_arr_GWOWOAlinear = []
mod_arr_GWOWOAlinear = []
bst_arr_GWOWOAlinear = []
C_S_arr_GWOWOAlinear = []
P_S_arr_GWOWOAlinear = []

for i in range(0,20):
    print("GWOWOAlinear",i+1, end=" ")
    best_community, NMI, modularity,CS, PS = GWOWOAlinear(nodes,edges)
    NMI_arr_GWOWOAlinear.append(NMI)
    mod_arr_GWOWOAlinear.append(modularity)
    bst_arr_GWOWOAlinear.append(best_community)
    C_S_arr_GWOWOAlinear.append(CS)
    P_S_arr_GWOWOAlinear.append(PS)
    

NMI_avg_GWOWOAlinear = sum(NMI_arr_GWOWOAlinear)/len(NMI_arr_GWOWOAlinear)
mod_avg_GWOWOAlinear = sum(mod_arr_GWOWOAlinear)/len(mod_arr_GWOWOAlinear)
C_S_avg_GWOWOAlinear = sum(C_S_arr_GWOWOAlinear)/len(C_S_arr_GWOWOAlinear)
P_S_avg_GWOWOAlinear = sum(P_S_arr_GWOWOAlinear)/len(P_S_arr_GWOWOAlinear)


NMI_bst_GWOWOAlinear = max(NMI_arr_GWOWOAlinear)  #best
mod_bst_GWOWOAlinear = max(mod_arr_GWOWOAlinear)
C_S_bst_GWOWOAlinear = max(C_S_arr_GWOWOAlinear)
P_S_bst_GWOWOAlinear = max(P_S_arr_GWOWOAlinear)

NMI_lst_GWOWOAlinear = min(NMI_arr_GWOWOAlinear)  #least
mod_lst_GWOWOAlinear = min(mod_arr_GWOWOAlinear)
C_S_lst_GWOWOAlinear = min(C_S_arr_GWOWOAlinear)
P_S_lst_GWOWOAlinear = min(P_S_arr_GWOWOAlinear)




print("GWOWOAlinear")
print('NMI avg: ', NMI_avg_GWOWOAlinear)
print('NMI max: ', NMI_bst_GWOWOAlinear)
print('NMI min: ', NMI_lst_GWOWOAlinear)
print()
print('mod avg: ', mod_avg_GWOWOAlinear)
print('mod max: ', mod_bst_GWOWOAlinear)
print('mod min: ', mod_lst_GWOWOAlinear)
print()
print('C_S avg: ', C_S_avg_GWOWOAlinear)
print('C_S max: ', C_S_bst_GWOWOAlinear)
print('C_S min: ', C_S_lst_GWOWOAlinear)
print()
print('P_S avg: ', P_S_avg_GWOWOAlinear)
print('P_S max: ', P_S_bst_GWOWOAlinear)
print('P_S min: ', P_S_lst_GWOWOAlinear)
print("Done")

GWOWOAlinear 1 GWOWOAlinear 2 GWOWOAlinear 3 GWOWOAlinear 4 GWOWOAlinear 5 GWOWOAlinear 6 GWOWOAlinear 7 GWOWOAlinear 8 GWOWOAlinear 9 GWOWOAlinear 10 GWOWOAlinear 11 GWOWOAlinear 12 GWOWOAlinear 13 GWOWOAlinear 14 GWOWOAlinear 15 GWOWOAlinear 16 GWOWOAlinear 17 GWOWOAlinear 18 GWOWOAlinear 19 GWOWOAlinear 20 GWOWOAlinear
NMI avg:  0.9902855153836663
NMI max:  0.9974558636410645
NMI min:  0.9749311835042113

mod avg:  0.17902605891488635
mod max:  0.20447953149465325
mod min:  0.15419656799493184

C_S avg:  7.24966050898542
C_S max:  10.398092797091868
C_S min:  5.1965336956311825

P_S avg:  0.005377950580945833
P_S max:  0.0067607289829512045
P_S min:  0.004399195575666164
Done


In [13]:
def WOAGWOlinear(nodes,edges,population=20,generation=100,r=1.5,P_se=0.05, P_su = 0.05, P_rs = 0.1):
    graph=nx.Graph() 
    graph.add_nodes_from(nodes) #adds nodes
    graph.add_edges_from(edges) #add edges
    Adj = nx.adjacency_matrix(graph) 
    nodes_length = len(graph.nodes())
#     nx.draw(graph, with_labels=True,node_color = "red")
#     plt.show()
    d = {"chrom":[generate_chrom(nodes_length,Adj) for n in range(population)]}
    dframe = pd.DataFrame(data= d)
    dframe["subsets"] = dframe["chrom"].apply(find_subsets)
    dframe["community_score"]=dframe.apply(lambda x: community_score(x["chrom"],x["subsets"],r,Adj),axis=1)
    gen = 0
    globalbest=[]
    globalbest = dframe.sort_values("community_score",ascending=False).index[0]
    globalbest = dframe.loc[globalbest]
    population_count = population
    while gen < generation:
        for i in dframe.index:
            
            r = np.random.rand()
            a = (2-2*gen)/generation
            M = (2*r-1)*a 
            if r < 0.5:
                if abs(M) < 1:
                    dframe.loc[i] = ShrinkingEncircling(dframe.loc[i],P_se, Adj)
                elif abs(M)>=1:
                    dframe.loc[i] = RandomSearchingOperation(dframe.loc[i],P_rs,Adj)
            elif r>=0.5:
                dframe.loc[i] = SpiralUpdatingOperation(dframe.loc[i],P_su,globalbest,Adj)
        tempbest = dframe.sort_values("community_score",ascending=False).index[0]
        tempbest = dframe.loc[tempbest]
        
        if globalbest["community_score"]<tempbest["community_score"]:
            globalbest = tempbest
        for i in range(int(np.floor(population/10))):
            elites = dframe.sort_values("community_score",ascending=True)[int(np.floor(population/10)):]
            best3 = dframe.sort_values("community_score", ascending=False)[0:3]
            xalpha = dframe.loc[best3.index[0]]['chrom']
            xbeta = dframe.loc[best3.index[1]]['chrom']
            xgamma = dframe.loc[best3.index[2]]['chrom']
            a = 2*(1-(gen/generation))
            for index , wolf in elites.iterrows():
                A1 = (2*a*np.random.rand()) - a
                C1 = 2*np.random.rand()
                Dalpha = abs(C1*xalpha - wolf['chrom'])
                X1 = abs(xalpha - (A1*Dalpha))
                A2 = (2*a*np.random.rand()) - a
                C2 = 2*np.random.rand()
                Dbeta = abs(C2*xbeta - wolf['chrom'])
                X2 = abs(xbeta - (A2*Dbeta))
                A3 = (2*a*np.random.rand()) - a
                C3 = 2*np.random.rand()
                Dgamma = abs(C3*xgamma - wolf['chrom'])
                X3 = abs(xgamma - (A3*Dgamma))
                Xnew = (X1+X2+X3)/3
                Xnew = np.array([int(i) for i in Xnew])
                for i in range(0,len(Xnew)):
                    if Xnew[i] < 0 or Xnew[i] >= Adj.shape[0] or Adj[i,Xnew[i]]==0:
                        Xnew[i] = xalpha[i]
                subsetnew = find_subsets(Xnew)
                fscorenew = community_score(Xnew,subsetnew, r, Adj)
                dframe.loc[population_count]=[Xnew,subsetnew,fscorenew]
                population_count += 1
        
        dfsorted = dframe.sort_values("community_score",ascending=False)
        to_drop = dfsorted.index[population:]
        dframe.drop(to_drop,inplace=True)
        gen +=1   
    sorted_df = dframe.sort_values("community_score",ascending=False).index[0]
    res = dframe.loc[sorted_df]
    res = res['chrom']
    res_subsets = find_subsets(res)
    nodes_subsets = res_subsets
    CS,PS = score_calc(res, nodes_subsets, r, Adj)
    nodes_list = list(graph.nodes())
    result = []
    for subs in nodes_subsets:
        subset = []
        for n in subs:
            subset.append(nodes_list[n])
        result.append(subset)
    NMI = 0
    clu = dframe.loc[sorted_df]
    clu = clu['chrom']
    clu = np.array(clu)
    for index, target in dframe.iterrows():
        temp = np.array(target['chrom'])
        x = computeNMI(clu,temp)
        NMI += x
    NMI /= len(dframe)
    modularity = nx_comm.modularity(graph, result)
    return result, NMI, modularity, CS, PS
print("WOAGWOlinear ready to use")

WOAGWOlinear ready to use


In [14]:
NMI_arr_WOAGWOlinear = []
mod_arr_WOAGWOlinear = []
bst_arr_WOAGWOlinear = []
C_S_arr_WOAGWOlinear = []
P_S_arr_WOAGWOlinear = []

for i in range(0,20):
    print("WOAGWOlinear",i+1, end=" ")
    best_community, NMI, modularity,CS, PS = WOAGWOlinear(nodes,edges)
    NMI_arr_WOAGWOlinear.append(NMI)
    mod_arr_WOAGWOlinear.append(modularity)
    bst_arr_WOAGWOlinear.append(best_community)
    C_S_arr_WOAGWOlinear.append(CS)
    P_S_arr_WOAGWOlinear.append(PS)
    

NMI_avg_WOAGWOlinear = sum(NMI_arr_WOAGWOlinear)/len(NMI_arr_WOAGWOlinear)
mod_avg_WOAGWOlinear = sum(mod_arr_WOAGWOlinear)/len(mod_arr_WOAGWOlinear)
C_S_avg_WOAGWOlinear = sum(C_S_arr_WOAGWOlinear)/len(C_S_arr_WOAGWOlinear)
P_S_avg_WOAGWOlinear = sum(P_S_arr_WOAGWOlinear)/len(P_S_arr_WOAGWOlinear)


NMI_bst_WOAGWOlinear = max(NMI_arr_WOAGWOlinear)  #best
mod_bst_WOAGWOlinear = max(mod_arr_WOAGWOlinear)
C_S_bst_WOAGWOlinear = max(C_S_arr_WOAGWOlinear)
P_S_bst_WOAGWOlinear = max(P_S_arr_WOAGWOlinear)

NMI_lst_WOAGWOlinear = min(NMI_arr_WOAGWOlinear)  #least
mod_lst_WOAGWOlinear = min(mod_arr_WOAGWOlinear)
C_S_lst_WOAGWOlinear = min(C_S_arr_WOAGWOlinear)
P_S_lst_WOAGWOlinear = min(P_S_arr_WOAGWOlinear)




print("WOAGWOlinear")
print('NMI avg: ', NMI_avg_WOAGWOlinear)
print('NMI max: ', NMI_bst_WOAGWOlinear)
print('NMI min: ', NMI_lst_WOAGWOlinear)
print()
print('mod avg: ', mod_avg_WOAGWOlinear)
print('mod max: ', mod_bst_WOAGWOlinear)
print('mod min: ', mod_lst_WOAGWOlinear)
print()
print('C_S avg: ', C_S_avg_WOAGWOlinear)
print('C_S max: ', C_S_bst_WOAGWOlinear)
print('C_S min: ', C_S_lst_WOAGWOlinear)
print()
print('P_S avg: ', P_S_avg_WOAGWOlinear)
print('P_S max: ', P_S_bst_WOAGWOlinear)
print('P_S min: ', P_S_lst_WOAGWOlinear)
print("Done")

WOAGWOlinear 1 WOAGWOlinear 2 WOAGWOlinear 3 WOAGWOlinear 4 WOAGWOlinear 5 WOAGWOlinear 6 WOAGWOlinear 7 WOAGWOlinear 8 WOAGWOlinear 9 WOAGWOlinear 10 WOAGWOlinear 11 WOAGWOlinear 12 WOAGWOlinear 13 WOAGWOlinear 14 WOAGWOlinear 15 WOAGWOlinear 16 WOAGWOlinear 17 WOAGWOlinear 18 WOAGWOlinear 19 WOAGWOlinear 20 WOAGWOlinear
NMI avg:  0.9985466474360718
NMI max:  1.0
NMI min:  0.9924520995111263

mod avg:  0.17084750269615168
mod max:  0.2134769444603353
mod min:  0.14856060461598114

C_S avg:  7.489236368823152
C_S max:  15.710040056135384
C_S min:  4.228650927370757

P_S avg:  0.005525609779882766
P_S max:  0.00777000777000777
P_S min:  0.004184704184704185
Done


In [15]:
def GWOWOAMix(nodes,edges,population=20,generation=100,r=1.5,P_se=0.05, P_su = 0.05, P_rs = 0.1):
    graph=nx.Graph() 
    graph.add_nodes_from(nodes) #adds nodes
    graph.add_edges_from(edges) #add edges
    Adj = nx.adjacency_matrix(graph) 
    nodes_length = len(graph.nodes())
#     nx.draw(graph, with_labels=True,node_color = "red")
#     plt.show()
    d = {"chrom":[generate_chrom(nodes_length,Adj) for n in range(population)]}
    dframe = pd.DataFrame(data= d)
    dframe["subsets"] = dframe["chrom"].apply(find_subsets)
    dframe["community_score"]=dframe.apply(lambda x: community_score(x["chrom"],x["subsets"],r,Adj),axis=1)
    gen = 0
    globalbest=[]
    globalbest = dframe.sort_values("community_score",ascending=False).index[0]
    globalbest = dframe.loc[globalbest]
    population_count = population
    while gen < generation:
       
        for i in range(int(np.floor(population/10))):
            elites = dframe.sort_values("community_score",ascending=True)[int(np.floor(population/10)):]
            best3 = dframe.sort_values("community_score", ascending=False)[0:3]
            xalpha = dframe.loc[best3.index[0]]['chrom']
            xbeta = dframe.loc[best3.index[1]]['chrom']
            xgamma = dframe.loc[best3.index[2]]['chrom']
            
            r = np.random.rand()
            a = (2-2*gen)/generation
            M = (2*r-1)*a 
            if r < 0.5:
                if abs(M) < 1:
                    dframe.loc[i] = ShrinkingEncircling(dframe.loc[i],P_se, Adj)
                elif abs(M)>=1:
                    dframe.loc[i] = RandomSearchingOperation(dframe.loc[i],P_rs,Adj)

            elif r>=0.5:
#                 dframe.loc[i] = SpiralUpdatingOperation(dframe.loc[i],P_su,globalbest,Adj)
                dframe.loc[i] = gwolagao(xalpha,xbeta,xgamma,dframe.loc[i], Adj, a)
        tempbest = dframe.sort_values("community_score",ascending=False).index[0]
        tempbest = dframe.loc[tempbest]
        
        if globalbest["community_score"]<tempbest["community_score"]:
            globalbest = tempbest
                           
        gen +=1   
    sorted_df = dframe.sort_values("community_score",ascending=False).index[0]
    res = dframe.loc[sorted_df]
    res = res['chrom']
    res_subsets = find_subsets(res)
    nodes_subsets = res_subsets
    CS, PS = score_calc(res, nodes_subsets, r, Adj)
    nodes_list = list(graph.nodes())
    result = []
    for subs in nodes_subsets:
        subset = []
        for n in subs:
            subset.append(nodes_list[n])
        result.append(subset)
    NMI = 0
    clu = dframe.loc[sorted_df]
    clu = clu['chrom']
    clu = np.array(clu)
    for index, target in dframe.iterrows():
        temp = np.array(target['chrom'])
        x = computeNMI(clu,temp)
        NMI += x
    NMI /= len(dframe)
    modularity = nx_comm.modularity(graph, result)
    return result, NMI, modularity, CS, PS
print("GWOWOAMix ready to use")

GWOWOAMix ready to use


In [16]:
NMI_arr_GWOWOAMix = []
mod_arr_GWOWOAMix = []
bst_arr_GWOWOAMix = []
C_S_arr_GWOWOAMix = []
P_S_arr_GWOWOAMix = []

for i in range(0,20):
    print("GWOWOAMix",i+1, end=" ")
    best_community, NMI, modularity,CS, PS = GWOWOAMix(nodes,edges)
    NMI_arr_GWOWOAMix.append(NMI)
    mod_arr_GWOWOAMix.append(modularity)
    bst_arr_GWOWOAMix.append(best_community)
    C_S_arr_GWOWOAMix.append(CS)
    P_S_arr_GWOWOAMix.append(PS)
    

NMI_avg_GWOWOAMix = sum(NMI_arr_GWOWOAMix)/len(NMI_arr_GWOWOAMix)
mod_avg_GWOWOAMix = sum(mod_arr_GWOWOAMix)/len(mod_arr_GWOWOAMix)
C_S_avg_GWOWOAMix = sum(C_S_arr_GWOWOAMix)/len(C_S_arr_GWOWOAMix)
P_S_avg_GWOWOAMix = sum(P_S_arr_GWOWOAMix)/len(P_S_arr_GWOWOAMix)


NMI_bst_GWOWOAMix = max(NMI_arr_GWOWOAMix)  #best
mod_bst_GWOWOAMix = max(mod_arr_GWOWOAMix)
C_S_bst_GWOWOAMix = max(C_S_arr_GWOWOAMix)
P_S_bst_GWOWOAMix = max(P_S_arr_GWOWOAMix)

NMI_lst_GWOWOAMix = min(NMI_arr_GWOWOAMix)  #least
mod_lst_GWOWOAMix = min(mod_arr_GWOWOAMix)
C_S_lst_GWOWOAMix = min(C_S_arr_GWOWOAMix)
P_S_lst_GWOWOAMix = min(P_S_arr_GWOWOAMix)




print("GWOWOAMix")
print('NMI avg: ', NMI_avg_GWOWOAMix)
print('NMI max: ', NMI_bst_GWOWOAMix)
print('NMI min: ', NMI_lst_GWOWOAMix)
print()
print('mod avg: ', mod_avg_GWOWOAMix)
print('mod max: ', mod_bst_GWOWOAMix)
print('mod min: ', mod_lst_GWOWOAMix)
print()
print('C_S avg: ', C_S_avg_GWOWOAMix)
print('C_S max: ', C_S_bst_GWOWOAMix)
print('C_S min: ', C_S_lst_GWOWOAMix)
print()
print('P_S avg: ', P_S_avg_GWOWOAMix)
print('P_S max: ', P_S_bst_GWOWOAMix)
print('P_S min: ', P_S_lst_GWOWOAMix)
print("Done")

GWOWOAMix 1 GWOWOAMix 2 GWOWOAMix 3 GWOWOAMix 4 GWOWOAMix 5 GWOWOAMix 6 GWOWOAMix 7 GWOWOAMix 8 GWOWOAMix 9 GWOWOAMix 10 GWOWOAMix 11 GWOWOAMix 12 GWOWOAMix 13 GWOWOAMix 14 GWOWOAMix 15 GWOWOAMix 16 GWOWOAMix 17 GWOWOAMix 18 GWOWOAMix 19 GWOWOAMix 20 GWOWOAMix
NMI avg:  0.872847372321228
NMI max:  0.884877298897945
NMI min:  0.85364407658728

mod avg:  0.1558951965065502
mod max:  0.20290315677512719
mod min:  0.09575628907830808

C_S avg:  22.666165411237632
C_S max:  59.31816572648387
C_S min:  7.793847625694167

P_S avg:  0.019057837247469917
P_S max:  0.02839506172839506
P_S min:  0.011507936507936509
Done


In [17]:
def PW1(nodes,edges,population=20,generation=100,P_se=0.05, P_su = 0.05, P_rs = 0.1, r=1.5):
    
#     common part in both
    
    graph=nx.Graph() 
    graph.add_nodes_from(nodes) #adds nodes
    graph.add_edges_from(edges) #add edges
    Adj = nx.adjacency_matrix(graph) 
    nodes_length = len(graph.nodes())
#     nx.draw(graph, with_labels=True,node_color = "red")
#     plt.show()
    d = {"chrom":[generate_chrom(nodes_length,Adj) for n in range(population)]}
    dframe = pd.DataFrame(data= d)
    dframe["subsets"] = dframe["chrom"].apply(find_subsets)
    dframe["community_score"]=dframe.apply(lambda x: community_score(x["chrom"],x["subsets"],1.5,Adj),axis=1)
    population_count = population
    gen = 0
    
#     velocity and variables of pso
    
    W = 0.8
    c1 = 0.3
    c2 = 0.7
    V = [np.random.randint(-nodes_length/3, nodes_length/3)]*len(dframe)
    dframe["vel"] = V
    
#     take global best and find it

    globalbest = dframe.sort_values("community_score",ascending=False).index[0]
    globalbest = dframe.loc[globalbest]
    
    tempbest = globalbest
    while gen < generation:
       
        for i in range(int(np.floor(population/10))):
#         for i in range(population):
            
            
            
            
            
            r = np.random.rand()
            a = (2-2*gen)/generation
            M = (2*r-1)*a 
            dtemp = dframe.loc[i]
            
#             dtemp = ChangingVelocity(dtemp, globalbest, tempbest, Adj, W, c1, c2, r)
#             dtemp = ShrinkingEncircling(dtemp,P_se, Adj)
#             dtemp = RandomSearchingOperation(dtemp,P_rs,Adj)
#             dtemp = SpiralUpdatingOperation(dtemp,P_su,globalbest,Adj)
            
            
            if r < 0.5:
                if abs(M) < 1:
#                     dtemp = ShrinkingEncircling(dtemp,P_se, Adj)
                    dtemp = ChangingVelocity(dtemp, globalbest, tempbest, Adj, W, c1, c2, r)
                    
                elif abs(M)>=1:
                    dtemp = RandomSearchingOperation(dtemp,P_rs,Adj)
#                     dtemp = ChangingVelocity(dtemp, globalbest, tempbest, Adj, W, c1, c2, r)
                    
            elif r>=0.5:
                dtemp = SpiralUpdatingOperation(dtemp,P_su,globalbest,Adj)
#                 dtemp = ChangingVelocity(dtemp, globalbest, tempbest, Adj, W, c1, c2, r)

                
# #                 dtemp = ChangingVelocity(dtemp, globalbest, tempbest, Adj, W, c1, c2, r)
            
#                       ..............
            if(dtemp['community_score']>dframe.loc[i]['community_score']):
                dframe.at[i,'subsets'] = dtemp['subsets']
                dframe.at[i,'chrom'] = dtemp['chrom']
                dframe.at[i,'community_score'] = dtemp['community_score']
            dframe.at[i, 'vel'] = dtemp['vel']
            
            
            
          
        
        
#         update global
        tempbest = dframe.sort_values("community_score",ascending=False).index[0]
        tempbest = dframe.loc[tempbest]
        
        if globalbest["community_score"]<tempbest["community_score"]:
            globalbest = tempbest
                           
        gen +=1   
            
            
#     end copy
    res = globalbest
#     score = res["community_score"]
    nodes_subsets = res["subsets"]
    res = res['chrom']
    CS, PS = score_calc(res, nodes_subsets, r, Adj)
#     istrue = check_res(res,Adj)
#     print(istrue)
    nodes_list = list(graph.nodes())
    result = []
    for subs in nodes_subsets:
        subset = []
        for n in subs:
            subset.append(nodes_list[n])
        result.append(subset)
    NMI = 0
    clu = globalbest
    clu = clu['chrom']
    clu = np.array(clu)
    for index, target in dframe.iterrows():
        temp = np.array(target['chrom'])
        x = computeNMI(clu,temp)
        NMI += x
    NMI /= len(dframe)
    modularity = nx_comm.modularity(graph, result)
    return result, NMI, modularity, CS, PS
print("PW1 loaded to use")

PW1 loaded to use


In [18]:
NMI_arr_PW1 = []
mod_arr_PW1 = []
bst_arr_PW1 = []
C_S_arr_PW1 = []
P_S_arr_PW1 = []

for i in range(0,20):
    print("PW1",i+1, end=" ")
    best_community, NMI, modularity,CS, PS = PW1(nodes,edges)
    NMI_arr_PW1.append(NMI)
    mod_arr_PW1.append(modularity)
    bst_arr_PW1.append(best_community)
    C_S_arr_PW1.append(CS)
    P_S_arr_PW1.append(PS)
    

NMI_avg_PW1 = sum(NMI_arr_PW1)/len(NMI_arr_PW1)
mod_avg_PW1 = sum(mod_arr_PW1)/len(mod_arr_PW1)
C_S_avg_PW1 = sum(C_S_arr_PW1)/len(C_S_arr_PW1)
P_S_avg_PW1 = sum(P_S_arr_PW1)/len(P_S_arr_PW1)


NMI_bst_PW1 = max(NMI_arr_PW1)  #best
mod_bst_PW1 = max(mod_arr_PW1)
C_S_bst_PW1 = max(C_S_arr_PW1)
P_S_bst_PW1 = max(P_S_arr_PW1)

NMI_lst_PW1 = min(NMI_arr_PW1)  #least
mod_lst_PW1 = min(mod_arr_PW1)
C_S_lst_PW1 = min(C_S_arr_PW1)
P_S_lst_PW1 = min(P_S_arr_PW1)




print("PW1")
print('NMI avg: ', NMI_avg_PW1)
print('NMI max: ', NMI_bst_PW1)
print('NMI min: ', NMI_lst_PW1)
print()
print('mod avg: ', mod_avg_PW1)
print('mod max: ', mod_bst_PW1)
print('mod min: ', mod_lst_PW1)
print()
print('C_S avg: ', C_S_avg_PW1)
print('C_S max: ', C_S_bst_PW1)
print('C_S min: ', C_S_lst_PW1)
print()
print('P_S avg: ', P_S_avg_PW1)
print('P_S max: ', P_S_bst_PW1)
print('P_S min: ', P_S_lst_PW1)
print("Done")

PW1 1 PW1 2 PW1 3 PW1 4 PW1 5 PW1 6 PW1 7 PW1 8 PW1 9 PW1 10 PW1 11 PW1 12 PW1 13 PW1 14 PW1 15 PW1 16 PW1 17 PW1 18 PW1 19 PW1 20 PW1
NMI avg:  0.866670057580429
NMI max:  0.8769500646454507
NMI min:  0.8571053575154298

mod avg:  0.08019059514502011
mod max:  0.2231184675264689
mod min:  0.0028052689901242366

C_S avg:  86.44999452987471
C_S max:  437.0437977971797
C_S min:  18.393386672793582

P_S avg:  0.3292064899298927
P_S max:  1.92570479228374
P_S min:  0.01962809917355372
Done


In [19]:
def PW2(nodes,edges,population=20,generation=100,P_se=0.05, P_su = 0.05, P_rs = 0.1, r=1.5):
    
#     common part in both
    
    graph=nx.Graph() 
    graph.add_nodes_from(nodes) #adds nodes
    graph.add_edges_from(edges) #add edges
    Adj = nx.adjacency_matrix(graph) 
    nodes_length = len(graph.nodes())
#     nx.draw(graph, with_labels=True,node_color = "red")
#     plt.show()
    d = {"chrom":[generate_chrom(nodes_length,Adj) for n in range(population)]}
    dframe = pd.DataFrame(data= d)
    dframe["subsets"] = dframe["chrom"].apply(find_subsets)
    dframe["community_score"]=dframe.apply(lambda x: community_score(x["chrom"],x["subsets"],1.5,Adj),axis=1)
    population_count = population
    gen = 0
    
#     velocity and variables of pso
    
    W = 0.8
    c1 = 0.3
    c2 = 0.7
    V = [np.random.randint(-nodes_length/3, nodes_length/3)]*len(dframe)
    dframe["vel"] = V
    
#     take global best and find it

    globalbest = dframe.sort_values("community_score",ascending=False).index[0]
    globalbest = dframe.loc[globalbest]
    
    tempbest = globalbest
    while gen < generation:
       
        for i in range(int(np.floor(population/10))):
#         for i in range(population):
            
            
            
            
            
            r = np.random.rand()
            a = (2-2*gen)/generation
            M = (2*r-1)*a 
            dtemp = dframe.loc[i]
            
#             dtemp = ChangingVelocity(dtemp, globalbest, tempbest, Adj, W, c1, c2, r)
#             dtemp = ShrinkingEncircling(dtemp,P_se, Adj)
#             dtemp = RandomSearchingOperation(dtemp,P_rs,Adj)
#             dtemp = SpiralUpdatingOperation(dtemp,P_su,globalbest,Adj)
            
            
            if r < 0.5:
                if abs(M) < 1:
                    dtemp = ShrinkingEncircling(dtemp,P_se, Adj)
#                     dtemp = ChangingVelocity(dtemp, globalbest, tempbest, Adj, W, c1, c2, r)
                    
                elif abs(M)>=1:
#                     dtemp = RandomSearchingOperation(dtemp,P_rs,Adj)
                    dtemp = ChangingVelocity(dtemp, globalbest, tempbest, Adj, W, c1, c2, r)
                    
            elif r>=0.5:
                dtemp = SpiralUpdatingOperation(dtemp,P_su,globalbest,Adj)
#                 dtemp = ChangingVelocity(dtemp, globalbest, tempbest, Adj, W, c1, c2, r)

                
# #                 dtemp = ChangingVelocity(dtemp, globalbest, tempbest, Adj, W, c1, c2, r)
            
#                       ..............
            if(dtemp['community_score']>dframe.loc[i]['community_score']):
                dframe.at[i,'subsets'] = dtemp['subsets']
                dframe.at[i,'chrom'] = dtemp['chrom']
                dframe.at[i,'community_score'] = dtemp['community_score']
            dframe.at[i, 'vel'] = dtemp['vel']
            
            
            
          
        
        
#         update global
        tempbest = dframe.sort_values("community_score",ascending=False).index[0]
        tempbest = dframe.loc[tempbest]
        
        if globalbest["community_score"]<tempbest["community_score"]:
            globalbest = tempbest
                           
        gen +=1   
            
            
#     end copy
    res = globalbest
#     score = res["community_score"]
    nodes_subsets = res["subsets"]
    res = res['chrom']
    CS, PS = score_calc(res, nodes_subsets, r, Adj)
#     istrue = check_res(res,Adj)
#     print(istrue)
    nodes_list = list(graph.nodes())
    result = []
    for subs in nodes_subsets:
        subset = []
        for n in subs:
            subset.append(nodes_list[n])
        result.append(subset)
    NMI = 0
    clu = globalbest
    clu = clu['chrom']
    clu = np.array(clu)
    for index, target in dframe.iterrows():
        temp = np.array(target['chrom'])
        x = computeNMI(clu,temp)
        NMI += x
    NMI /= len(dframe)
    modularity = nx_comm.modularity(graph, result)
    return result, NMI, modularity, CS, PS
print("PW2 loaded to use")

PW2 loaded to use


In [20]:
NMI_arr_PW2 = []
mod_arr_PW2 = []
bst_arr_PW2 = []
C_S_arr_PW2 = []
P_S_arr_PW2 = []

for i in range(0,20):
    print("PW2",i+1, end=" ")
    best_community, NMI, modularity,CS, PS = PW2(nodes,edges)
    NMI_arr_PW2.append(NMI)
    mod_arr_PW2.append(modularity)
    bst_arr_PW2.append(best_community)
    C_S_arr_PW2.append(CS)
    P_S_arr_PW2.append(PS)
    

NMI_avg_PW2 = sum(NMI_arr_PW2)/len(NMI_arr_PW2)
mod_avg_PW2 = sum(mod_arr_PW2)/len(mod_arr_PW2)
C_S_avg_PW2 = sum(C_S_arr_PW2)/len(C_S_arr_PW2)
P_S_avg_PW2 = sum(P_S_arr_PW2)/len(P_S_arr_PW2)


NMI_bst_PW2 = max(NMI_arr_PW2)  #best
mod_bst_PW2 = max(mod_arr_PW2)
C_S_bst_PW2 = max(C_S_arr_PW2)
P_S_bst_PW2 = max(P_S_arr_PW2)

NMI_lst_PW2 = min(NMI_arr_PW2)  #least
mod_lst_PW2 = min(mod_arr_PW2)
C_S_lst_PW2 = min(C_S_arr_PW2)
P_S_lst_PW2 = min(P_S_arr_PW2)




print("PW2")
print('NMI avg: ', NMI_avg_PW2)
print('NMI max: ', NMI_bst_PW2)
print('NMI min: ', NMI_lst_PW2)
print()
print('mod avg: ', mod_avg_PW2)
print('mod max: ', mod_bst_PW2)
print('mod min: ', mod_lst_PW2)
print()
print('C_S avg: ', C_S_avg_PW2)
print('C_S max: ', C_S_bst_PW2)
print('C_S min: ', C_S_lst_PW2)
print()
print('P_S avg: ', P_S_avg_PW2)
print('P_S max: ', P_S_bst_PW2)
print('P_S min: ', P_S_lst_PW2)
print("Done")

PW2 1 PW2 2 PW2 3 PW2 4 PW2 5 PW2 6 PW2 7 PW2 8 PW2 9 PW2 10 PW2 11 PW2 12 PW2 13 PW2 14 PW2 15 PW2 16 PW2 17 PW2 18 PW2 19 PW2 20 PW2
NMI avg:  0.8072805862901055
NMI max:  0.8422142944575665
NMI min:  0.7660506523695788

mod avg:  0.036428981564467125
mod max:  0.13038991967692787
mod min:  0.0

C_S avg:  223.04201989933458
C_S max:  772.4500209085426
C_S min:  35.904561427293224

P_S avg:  33.32132770623483
P_S max:  131.0
P_S min:  0.05555555555555555
Done


In [21]:
def PW3(nodes,edges,population=20,generation=100,P_se=0.05, P_su = 0.05, P_rs = 0.1, r=1.5):
    
#     common part in both
    
    graph=nx.Graph() 
    graph.add_nodes_from(nodes) #adds nodes
    graph.add_edges_from(edges) #add edges
    Adj = nx.adjacency_matrix(graph) 
    nodes_length = len(graph.nodes())
#     nx.draw(graph, with_labels=True,node_color = "red")
#     plt.show()
    d = {"chrom":[generate_chrom(nodes_length,Adj) for n in range(population)]}
    dframe = pd.DataFrame(data= d)
    dframe["subsets"] = dframe["chrom"].apply(find_subsets)
    dframe["community_score"]=dframe.apply(lambda x: community_score(x["chrom"],x["subsets"],1.5,Adj),axis=1)
    population_count = population
    gen = 0
    
#     velocity and variables of pso
    
    W = 0.8
    c1 = 0.3
    c2 = 0.7
    V = [np.random.randint(-nodes_length/3, nodes_length/3)]*len(dframe)
    dframe["vel"] = V
    
#     take global best and find it

    globalbest = dframe.sort_values("community_score",ascending=False).index[0]
    globalbest = dframe.loc[globalbest]
    
    tempbest = globalbest
    while gen < generation:
       
        for i in range(int(np.floor(population/10))):
#         for i in range(population):
            
            
            
            
            
            r = np.random.rand()
            a = (2-2*gen)/generation
            M = (2*r-1)*a 
            dtemp = dframe.loc[i]
            
#             dtemp = ChangingVelocity(dtemp, globalbest, tempbest, Adj, W, c1, c2, r)
#             dtemp = ShrinkingEncircling(dtemp,P_se, Adj)
#             dtemp = RandomSearchingOperation(dtemp,P_rs,Adj)
#             dtemp = SpiralUpdatingOperation(dtemp,P_su,globalbest,Adj)
            
            
            if r < 0.5:
                if abs(M) < 1:
                    dtemp = ShrinkingEncircling(dtemp,P_se, Adj)
#                     dtemp = ChangingVelocity(dtemp, globalbest, tempbest, Adj, W, c1, c2, r)
                    
                elif abs(M)>=1:
                    dtemp = RandomSearchingOperation(dtemp,P_rs,Adj)
#                     dtemp = ChangingVelocity(dtemp, globalbest, tempbest, Adj, W, c1, c2, r)
                    
            elif r>=0.5:
#                 dtemp = SpiralUpdatingOperation(dtemp,P_su,globalbest,Adj)
                dtemp = ChangingVelocity(dtemp, globalbest, tempbest, Adj, W, c1, c2, r)

                
# #                 dtemp = ChangingVelocity(dtemp, globalbest, tempbest, Adj, W, c1, c2, r)
            
#                       ..............
            if(dtemp['community_score']>dframe.loc[i]['community_score']):
                dframe.at[i,'subsets'] = dtemp['subsets']
                dframe.at[i,'chrom'] = dtemp['chrom']
                dframe.at[i,'community_score'] = dtemp['community_score']
            dframe.at[i, 'vel'] = dtemp['vel']
            
            
            
          
        
        
#         update global
        tempbest = dframe.sort_values("community_score",ascending=False).index[0]
        tempbest = dframe.loc[tempbest]
        
        if globalbest["community_score"]<tempbest["community_score"]:
            globalbest = tempbest
                           
        gen +=1   
            
            
#     end copy
    res = globalbest
#     score = res["community_score"]
    nodes_subsets = res["subsets"]
    res = res['chrom']
    CS, PS = score_calc(res, nodes_subsets, r, Adj)
#     istrue = check_res(res,Adj)
#     print(istrue)
    nodes_list = list(graph.nodes())
    result = []
    for subs in nodes_subsets:
        subset = []
        for n in subs:
            subset.append(nodes_list[n])
        result.append(subset)
    NMI = 0
    clu = globalbest
    clu = clu['chrom']
    clu = np.array(clu)
    for index, target in dframe.iterrows():
        temp = np.array(target['chrom'])
        x = computeNMI(clu,temp)
        NMI += x
    NMI /= len(dframe)
    modularity = nx_comm.modularity(graph, result)
    return result, NMI, modularity, CS, PS
print("PW3 loaded to use")

PW3 loaded to use


In [22]:
NMI_arr_PW3 = []
mod_arr_PW3 = []
bst_arr_PW3 = []
C_S_arr_PW3 = []
P_S_arr_PW3 = []

for i in range(0,20):
    print("PW3",i+1, end=" ")
    best_community, NMI, modularity,CS, PS = PW3(nodes,edges)
    NMI_arr_PW3.append(NMI)
    mod_arr_PW3.append(modularity)
    bst_arr_PW3.append(best_community)
    C_S_arr_PW3.append(CS)
    P_S_arr_PW3.append(PS)
    

NMI_avg_PW3 = sum(NMI_arr_PW3)/len(NMI_arr_PW3)
mod_avg_PW3 = sum(mod_arr_PW3)/len(mod_arr_PW3)
C_S_avg_PW3 = sum(C_S_arr_PW3)/len(C_S_arr_PW3)
P_S_avg_PW3 = sum(P_S_arr_PW3)/len(P_S_arr_PW3)


NMI_bst_PW3 = max(NMI_arr_PW3)  #best
mod_bst_PW3 = max(mod_arr_PW3)
C_S_bst_PW3 = max(C_S_arr_PW3)
P_S_bst_PW3 = max(P_S_arr_PW3)

NMI_lst_PW3 = min(NMI_arr_PW3)  #least
mod_lst_PW3 = min(mod_arr_PW3)
C_S_lst_PW3 = min(C_S_arr_PW3)
P_S_lst_PW3 = min(P_S_arr_PW3)




print("PW3")
print('NMI avg: ', NMI_avg_PW3)
print('NMI max: ', NMI_bst_PW3)
print('NMI min: ', NMI_lst_PW3)
print()
print('mod avg: ', mod_avg_PW3)
print('mod max: ', mod_bst_PW3)
print('mod min: ', mod_lst_PW3)
print()
print('C_S avg: ', C_S_avg_PW3)
print('C_S max: ', C_S_bst_PW3)
print('C_S min: ', C_S_lst_PW3)
print()
print('P_S avg: ', P_S_avg_PW3)
print('P_S max: ', P_S_bst_PW3)
print('P_S min: ', P_S_lst_PW3)
print("Done")

PW3 1 PW3 2 PW3 3 PW3 4 PW3 5 PW3 6 PW3 7 PW3 8 PW3 9 PW3 10 PW3 11 PW3 12 PW3 13 PW3 14 PW3 15 PW3 16 PW3 17 PW3 18 PW3 19 PW3 20 PW3
NMI avg:  0.8229725322365888
NMI max:  0.8868888233782568
NMI min:  0.7920094389653654

mod avg:  0.045526930794183525
mod max:  0.20550078500918492
mod min:  0.0

C_S avg:  235.7488518252954
C_S max:  674.6860577444893
C_S min:  8.043536601250349

P_S avg:  65.58594409686927
P_S max:  131.0
P_S min:  0.011188811188811189
Done


In [23]:
def PWVel(nodes,edges,population=20,generation=100,P_se=0.05, P_su = 0.05, P_rs = 0.1, r=1.5):
    
#     common part in both
    
    graph=nx.Graph() 
    graph.add_nodes_from(nodes) #adds nodes
    graph.add_edges_from(edges) #add edges
    Adj = nx.adjacency_matrix(graph) 
    nodes_length = len(graph.nodes())
#     nx.draw(graph, with_labels=True,node_color = "red")
#     plt.show()
    d = {"chrom":[generate_chrom(nodes_length,Adj) for n in range(population)]}
    dframe = pd.DataFrame(data= d)
    dframe["subsets"] = dframe["chrom"].apply(find_subsets)
    dframe["community_score"]=dframe.apply(lambda x: community_score(x["chrom"],x["subsets"],1.5,Adj),axis=1)
    population_count = population
    gen = 0
    
#     velocity and variables of pso
    
    W = 0.8
    c1 = 0.3
    c2 = 0.7
    V = [np.random.randint(-nodes_length/3, nodes_length/3)]*len(dframe)
    dframe["vel"] = V
    
#     take global best and find it

    globalbest = dframe.sort_values("community_score",ascending=False).index[0]
    globalbest = dframe.loc[globalbest]
    
    tempbest = globalbest
    while gen < generation:
       
        for i in range(int(np.floor(population/10))):
#         for i in range(population):
            
            
            
            
            
            r = np.random.rand()
            a = (2-2*gen)/generation
            M = (2*r-1)*a 
            dtemp = dframe.loc[i]
            
            dtemp = ChangingVelocity(dtemp, globalbest, tempbest, Adj, W, c1, c2, r)
#             dtemp = ShrinkingEncircling(dtemp,P_se, Adj)
#             dtemp = RandomSearchingOperation(dtemp,P_rs,Adj)
#             dtemp = SpiralUpdatingOperation(dtemp,P_su,globalbest,Adj)
            
            
#             if r < 0.5:
#                 if abs(M) < 1:
#                     dtemp = ShrinkingEncircling(dtemp,P_se, Adj)
# #                     dtemp = ChangingVelocity(dtemp, globalbest, tempbest, Adj, W, c1, c2, r)
                    
#                 elif abs(M)>=1:
#                     dtemp = RandomSearchingOperation(dtemp,P_rs,Adj)
# #                     dtemp = ChangingVelocity(dtemp, globalbest, tempbest, Adj, W, c1, c2, r)
                    
#             elif r>=0.5:
# #                 dtemp = SpiralUpdatingOperation(dtemp,P_su,globalbest,Adj)
#                 dtemp = ChangingVelocity(dtemp, globalbest, tempbest, Adj, W, c1, c2, r)

                
# #                 dtemp = ChangingVelocity(dtemp, globalbest, tempbest, Adj, W, c1, c2, r)
            
#                       ..............
            if(dtemp['community_score']>dframe.loc[i]['community_score']):
                dframe.at[i,'subsets'] = dtemp['subsets']
                dframe.at[i,'chrom'] = dtemp['chrom']
                dframe.at[i,'community_score'] = dtemp['community_score']
            dframe.at[i, 'vel'] = dtemp['vel']
            
            
            
          
        
        
#         update global
        tempbest = dframe.sort_values("community_score",ascending=False).index[0]
        tempbest = dframe.loc[tempbest]
        
        if globalbest["community_score"]<tempbest["community_score"]:
            globalbest = tempbest
                           
        gen +=1   
            
            
#     end copy
    res = globalbest
#     score = res["community_score"]
    nodes_subsets = res["subsets"]
    res = res['chrom']
    CS, PS = score_calc(res, nodes_subsets, r, Adj)
#     istrue = check_res(res,Adj)
#     print(istrue)
    nodes_list = list(graph.nodes())
    result = []
    for subs in nodes_subsets:
        subset = []
        for n in subs:
            subset.append(nodes_list[n])
        result.append(subset)
    NMI = 0
    clu = globalbest
    clu = clu['chrom']
    clu = np.array(clu)
    for index, target in dframe.iterrows():
        temp = np.array(target['chrom'])
        x = computeNMI(clu,temp)
        NMI += x
    NMI /= len(dframe)
    modularity = nx_comm.modularity(graph, result)
    return result, NMI, modularity, CS, PS
print("PWVel loaded to use")

PWVel loaded to use


In [24]:
NMI_arr_PWVel = []
mod_arr_PWVel = []
bst_arr_PWVel = []
C_S_arr_PWVel = []
P_S_arr_PWVel = []

for i in range(0,20):
    print("PWVel",i+1, end=" ")
    best_community, NMI, modularity,CS, PS = PWVel(nodes,edges)
    NMI_arr_PWVel.append(NMI)
    mod_arr_PWVel.append(modularity)
    bst_arr_PWVel.append(best_community)
    C_S_arr_PWVel.append(CS)
    P_S_arr_PWVel.append(PS)
    

NMI_avg_PWVel = sum(NMI_arr_PWVel)/len(NMI_arr_PWVel)
mod_avg_PWVel = sum(mod_arr_PWVel)/len(mod_arr_PWVel)
C_S_avg_PWVel = sum(C_S_arr_PWVel)/len(C_S_arr_PWVel)
P_S_avg_PWVel = sum(P_S_arr_PWVel)/len(P_S_arr_PWVel)


NMI_bst_PWVel = max(NMI_arr_PWVel)  #best
mod_bst_PWVel = max(mod_arr_PWVel)
C_S_bst_PWVel = max(C_S_arr_PWVel)
P_S_bst_PWVel = max(P_S_arr_PWVel)

NMI_lst_PWVel = min(NMI_arr_PWVel)  #least
mod_lst_PWVel = min(mod_arr_PWVel)
C_S_lst_PWVel = min(C_S_arr_PWVel)
P_S_lst_PWVel = min(P_S_arr_PWVel)




print("PWVel")
print('NMI avg: ', NMI_avg_PWVel)
print('NMI max: ', NMI_bst_PWVel)
print('NMI min: ', NMI_lst_PWVel)
print()
print('mod avg: ', mod_avg_PWVel)
print('mod max: ', mod_bst_PWVel)
print('mod min: ', mod_lst_PWVel)
print()
print('C_S avg: ', C_S_avg_PWVel)
print('C_S max: ', C_S_bst_PWVel)
print('C_S min: ', C_S_lst_PWVel)
print()
print('P_S avg: ', P_S_avg_PWVel)
print('P_S max: ', P_S_bst_PWVel)
print('P_S min: ', P_S_lst_PWVel)
print("Done")

PWVel 1 PWVel 2 PWVel 3 PWVel 4 PWVel 5 PWVel 6 PWVel 7 PWVel 8 PWVel 9 PWVel 10 PWVel 11 PWVel 12 PWVel 13 PWVel 14 PWVel 15 PWVel 16 PWVel 17 PWVel 18 PWVel 19 PWVel 20 PWVel
NMI avg:  0.8590002781164614
NMI max:  0.8721831325844212
NMI min:  0.822491525086328

mod avg:  0.08345288567681351
mod max:  0.16728111380196575
mod min:  0.0028052689901242366

C_S avg:  111.47951909362868
C_S max:  338.6430799619807
C_S min:  15.473427652367425

P_S avg:  3.2525729287962752
P_S max:  58.040211979384786
P_S min:  0.02686202686202686
Done


In [25]:
def WShrEnc(nodes,edges,population=20,generation=100,P_se=0.05, P_su = 0.05, P_rs = 0.1, r=1.5):
    
#     common part in both
    
    graph=nx.Graph() 
    graph.add_nodes_from(nodes) #adds nodes
    graph.add_edges_from(edges) #add edges
    Adj = nx.adjacency_matrix(graph) 
    nodes_length = len(graph.nodes())
#     nx.draw(graph, with_labels=True,node_color = "red")
#     plt.show()
    d = {"chrom":[generate_chrom(nodes_length,Adj) for n in range(population)]}
    dframe = pd.DataFrame(data= d)
    dframe["subsets"] = dframe["chrom"].apply(find_subsets)
    dframe["community_score"]=dframe.apply(lambda x: community_score(x["chrom"],x["subsets"],1.5,Adj),axis=1)
    population_count = population
    gen = 0
    
#     velocity and variables of pso
    
    W = 0.8
    c1 = 0.3
    c2 = 0.7
    V = [np.random.randint(-nodes_length/3, nodes_length/3)]*len(dframe)
    dframe["vel"] = V
    
#     take global best and find it

    globalbest = dframe.sort_values("community_score",ascending=False).index[0]
    globalbest = dframe.loc[globalbest]
    
    tempbest = globalbest
    while gen < generation:
       
        for i in range(int(np.floor(population/10))):
#         for i in range(population):
            
            
            
            
            
            r = np.random.rand()
            a = (2-2*gen)/generation
            M = (2*r-1)*a 
            dtemp = dframe.loc[i]
            
#             dtemp = ChangingVelocity(dtemp, globalbest, tempbest, Adj, W, c1, c2, r)
            dtemp = ShrinkingEncircling(dtemp,P_se, Adj)
#             dtemp = RandomSearchingOperation(dtemp,P_rs,Adj)
#             dtemp = SpiralUpdatingOperation(dtemp,P_su,globalbest,Adj)
            
            
#             if r < 0.5:
#                 if abs(M) < 1:
#                     dtemp = ShrinkingEncircling(dtemp,P_se, Adj)
# #                     dtemp = ChangingVelocity(dtemp, globalbest, tempbest, Adj, W, c1, c2, r)
                    
#                 elif abs(M)>=1:
#                     dtemp = RandomSearchingOperation(dtemp,P_rs,Adj)
# #                     dtemp = ChangingVelocity(dtemp, globalbest, tempbest, Adj, W, c1, c2, r)
                    
#             elif r>=0.5:
# #                 dtemp = SpiralUpdatingOperation(dtemp,P_su,globalbest,Adj)
#                 dtemp = ChangingVelocity(dtemp, globalbest, tempbest, Adj, W, c1, c2, r)

                
# #                 dtemp = ChangingVelocity(dtemp, globalbest, tempbest, Adj, W, c1, c2, r)
            
#                       ..............
            if(dtemp['community_score']>dframe.loc[i]['community_score']):
                dframe.at[i,'subsets'] = dtemp['subsets']
                dframe.at[i,'chrom'] = dtemp['chrom']
                dframe.at[i,'community_score'] = dtemp['community_score']
            dframe.at[i, 'vel'] = dtemp['vel']
            
            
            
          
        
        
#         update global
        tempbest = dframe.sort_values("community_score",ascending=False).index[0]
        tempbest = dframe.loc[tempbest]
        
        if globalbest["community_score"]<tempbest["community_score"]:
            globalbest = tempbest
                           
        gen +=1   
            
            
#     end copy
    res = globalbest
#     score = res["community_score"]
    nodes_subsets = res["subsets"]
    res = res['chrom']
    CS, PS = score_calc(res, nodes_subsets, r, Adj)
#     istrue = check_res(res,Adj)
#     print(istrue)
    nodes_list = list(graph.nodes())
    result = []
    for subs in nodes_subsets:
        subset = []
        for n in subs:
            subset.append(nodes_list[n])
        result.append(subset)
    NMI = 0
    clu = globalbest
    clu = clu['chrom']
    clu = np.array(clu)
    for index, target in dframe.iterrows():
        temp = np.array(target['chrom'])
        x = computeNMI(clu,temp)
        NMI += x
    NMI /= len(dframe)
    modularity = nx_comm.modularity(graph, result)
    return result, NMI, modularity, CS, PS
print("WShrEnc loaded to use")

WShrEnc loaded to use


In [26]:
NMI_arr_WShrEnc = []
mod_arr_WShrEnc = []
bst_arr_WShrEnc = []
C_S_arr_WShrEnc = []
P_S_arr_WShrEnc = []

for i in range(0,20):
    print("WShrEnc",i+1, end=" ")
    best_community, NMI, modularity,CS, PS = WShrEnc(nodes,edges)
    NMI_arr_WShrEnc.append(NMI)
    mod_arr_WShrEnc.append(modularity)
    bst_arr_WShrEnc.append(best_community)
    C_S_arr_WShrEnc.append(CS)
    P_S_arr_WShrEnc.append(PS)
    

NMI_avg_WShrEnc = sum(NMI_arr_WShrEnc)/len(NMI_arr_WShrEnc)
mod_avg_WShrEnc = sum(mod_arr_WShrEnc)/len(mod_arr_WShrEnc)
C_S_avg_WShrEnc = sum(C_S_arr_WShrEnc)/len(C_S_arr_WShrEnc)
P_S_avg_WShrEnc = sum(P_S_arr_WShrEnc)/len(P_S_arr_WShrEnc)


NMI_bst_WShrEnc = max(NMI_arr_WShrEnc)  #best
mod_bst_WShrEnc = max(mod_arr_WShrEnc)
C_S_bst_WShrEnc = max(C_S_arr_WShrEnc)
P_S_bst_WShrEnc = max(P_S_arr_WShrEnc)

NMI_lst_WShrEnc = min(NMI_arr_WShrEnc)  #least
mod_lst_WShrEnc = min(mod_arr_WShrEnc)
C_S_lst_WShrEnc = min(C_S_arr_WShrEnc)
P_S_lst_WShrEnc = min(P_S_arr_WShrEnc)




print("WShrEnc")
print('NMI avg: ', NMI_avg_WShrEnc)
print('NMI max: ', NMI_bst_WShrEnc)
print('NMI min: ', NMI_lst_WShrEnc)
print()
print('mod avg: ', mod_avg_WShrEnc)
print('mod max: ', mod_bst_WShrEnc)
print('mod min: ', mod_lst_WShrEnc)
print()
print('C_S avg: ', C_S_avg_WShrEnc)
print('C_S max: ', C_S_bst_WShrEnc)
print('C_S min: ', C_S_lst_WShrEnc)
print()
print('P_S avg: ', P_S_avg_WShrEnc)
print('P_S max: ', P_S_bst_WShrEnc)
print('P_S min: ', P_S_lst_WShrEnc)
print("Done")

WShrEnc 1 WShrEnc 2 WShrEnc 3 WShrEnc 4 WShrEnc 5 WShrEnc 6 WShrEnc 7 WShrEnc 8 WShrEnc 9 WShrEnc 10 WShrEnc 11 WShrEnc 12 WShrEnc 13 WShrEnc 14 WShrEnc 15 WShrEnc 16 WShrEnc 17 WShrEnc 18 WShrEnc 19 WShrEnc 20 WShrEnc
NMI avg:  0.822651160049834
NMI max:  0.8679468638899159
NMI min:  0.7406187575554146

mod avg:  0.150418883443616
mod max:  0.23306827355186466
mod min:  0.035482203280300086

C_S avg:  35.904835980442556
C_S max:  104.55845784441651
C_S min:  7.7718633162193544

P_S avg:  0.019259862711043272
P_S max:  0.030952380952380953
P_S min:  0.008484162895927603
Done


In [27]:
def WRanUpd(nodes,edges,population=20,generation=100,P_se=0.05, P_su = 0.05, P_rs = 0.1, r=1.5):
    
#     common part in both
    
    graph=nx.Graph() 
    graph.add_nodes_from(nodes) #adds nodes
    graph.add_edges_from(edges) #add edges
    Adj = nx.adjacency_matrix(graph) 
    nodes_length = len(graph.nodes())
#     nx.draw(graph, with_labels=True,node_color = "red")
#     plt.show()
    d = {"chrom":[generate_chrom(nodes_length,Adj) for n in range(population)]}
    dframe = pd.DataFrame(data= d)
    dframe["subsets"] = dframe["chrom"].apply(find_subsets)
    dframe["community_score"]=dframe.apply(lambda x: community_score(x["chrom"],x["subsets"],1.5,Adj),axis=1)
    population_count = population
    gen = 0
    
#     velocity and variables of pso
    
    W = 0.8
    c1 = 0.3
    c2 = 0.7
    V = [np.random.randint(-nodes_length/3, nodes_length/3)]*len(dframe)
    dframe["vel"] = V
    
#     take global best and find it

    globalbest = dframe.sort_values("community_score",ascending=False).index[0]
    globalbest = dframe.loc[globalbest]
    
    tempbest = globalbest
    while gen < generation:
       
        for i in range(int(np.floor(population/10))):
#         for i in range(population):
            
            
            
            
            
            r = np.random.rand()
            a = (2-2*gen)/generation
            M = (2*r-1)*a 
            dtemp = dframe.loc[i]
            
#             dtemp = ChangingVelocity(dtemp, globalbest, tempbest, Adj, W, c1, c2, r)
#             dtemp = ShrinkingEncircling(dtemp,P_se, Adj)
            dtemp = RandomSearchingOperation(dtemp,P_rs,Adj)
#             dtemp = SpiralUpdatingOperation(dtemp,P_su,globalbest,Adj)
            
            
#             if r < 0.5:
#                 if abs(M) < 1:
#                     dtemp = ShrinkingEncircling(dtemp,P_se, Adj)
# #                     dtemp = ChangingVelocity(dtemp, globalbest, tempbest, Adj, W, c1, c2, r)
                    
#                 elif abs(M)>=1:
#                     dtemp = RandomSearchingOperation(dtemp,P_rs,Adj)
# #                     dtemp = ChangingVelocity(dtemp, globalbest, tempbest, Adj, W, c1, c2, r)
                    
#             elif r>=0.5:
# #                 dtemp = SpiralUpdatingOperation(dtemp,P_su,globalbest,Adj)
#                 dtemp = ChangingVelocity(dtemp, globalbest, tempbest, Adj, W, c1, c2, r)

                
# #                 dtemp = ChangingVelocity(dtemp, globalbest, tempbest, Adj, W, c1, c2, r)
            
#                       ..............
            if(dtemp['community_score']>dframe.loc[i]['community_score']):
                dframe.at[i,'subsets'] = dtemp['subsets']
                dframe.at[i,'chrom'] = dtemp['chrom']
                dframe.at[i,'community_score'] = dtemp['community_score']
            dframe.at[i, 'vel'] = dtemp['vel']
            
            
            
          
        
        
#         update global
        tempbest = dframe.sort_values("community_score",ascending=False).index[0]
        tempbest = dframe.loc[tempbest]
        
        if globalbest["community_score"]<tempbest["community_score"]:
            globalbest = tempbest
                           
        gen +=1   
            
            
#     end copy
    res = globalbest
#     score = res["community_score"]
    nodes_subsets = res["subsets"]
    res = res['chrom']
    CS, PS = score_calc(res, nodes_subsets, r, Adj)
#     istrue = check_res(res,Adj)
#     print(istrue)
    nodes_list = list(graph.nodes())
    result = []
    for subs in nodes_subsets:
        subset = []
        for n in subs:
            subset.append(nodes_list[n])
        result.append(subset)
    NMI = 0
    clu = globalbest
    clu = clu['chrom']
    clu = np.array(clu)
    for index, target in dframe.iterrows():
        temp = np.array(target['chrom'])
        x = computeNMI(clu,temp)
        NMI += x
    NMI /= len(dframe)
    modularity = nx_comm.modularity(graph, result)
    return result, NMI, modularity, CS, PS
print("WRanUpd loaded to use")

WRanUpd loaded to use


In [28]:
NMI_arr_WRanUpd = []
mod_arr_WRanUpd = []
bst_arr_WRanUpd = []
C_S_arr_WRanUpd = []
P_S_arr_WRanUpd = []

for i in range(0,20):
    print("WRanUpd",i+1, end=" ")
    best_community, NMI, modularity,CS, PS = WRanUpd(nodes,edges)
    NMI_arr_WRanUpd.append(NMI)
    mod_arr_WRanUpd.append(modularity)
    bst_arr_WRanUpd.append(best_community)
    C_S_arr_WRanUpd.append(CS)
    P_S_arr_WRanUpd.append(PS)
    

NMI_avg_WRanUpd = sum(NMI_arr_WRanUpd)/len(NMI_arr_WRanUpd)
mod_avg_WRanUpd = sum(mod_arr_WRanUpd)/len(mod_arr_WRanUpd)
C_S_avg_WRanUpd = sum(C_S_arr_WRanUpd)/len(C_S_arr_WRanUpd)
P_S_avg_WRanUpd = sum(P_S_arr_WRanUpd)/len(P_S_arr_WRanUpd)


NMI_bst_WRanUpd = max(NMI_arr_WRanUpd)  #best
mod_bst_WRanUpd = max(mod_arr_WRanUpd)
C_S_bst_WRanUpd = max(C_S_arr_WRanUpd)
P_S_bst_WRanUpd = max(P_S_arr_WRanUpd)

NMI_lst_WRanUpd = min(NMI_arr_WRanUpd)  #least
mod_lst_WRanUpd = min(mod_arr_WRanUpd)
C_S_lst_WRanUpd = min(C_S_arr_WRanUpd)
P_S_lst_WRanUpd = min(P_S_arr_WRanUpd)




print("WRanUpd")
print('NMI avg: ', NMI_avg_WRanUpd)
print('NMI max: ', NMI_bst_WRanUpd)
print('NMI min: ', NMI_lst_WRanUpd)
print()
print('mod avg: ', mod_avg_WRanUpd)
print('mod max: ', mod_bst_WRanUpd)
print('mod min: ', mod_lst_WRanUpd)
print()
print('C_S avg: ', C_S_avg_WRanUpd)
print('C_S max: ', C_S_bst_WRanUpd)
print('C_S min: ', C_S_lst_WRanUpd)
print()
print('P_S avg: ', P_S_avg_WRanUpd)
print('P_S max: ', P_S_bst_WRanUpd)
print('P_S min: ', P_S_lst_WRanUpd)
print("Done")

WRanUpd 1 WRanUpd 2 WRanUpd 3 WRanUpd 4 WRanUpd 5 WRanUpd 6 WRanUpd 7 WRanUpd 8 WRanUpd 9 WRanUpd 10 WRanUpd 11 WRanUpd 12 WRanUpd 13 WRanUpd 14 WRanUpd 15 WRanUpd 16 WRanUpd 17 WRanUpd 18 WRanUpd 19 WRanUpd 20 WRanUpd
NMI avg:  0.8650132391353985
NMI max:  0.8758724907518239
NMI min:  0.8562021179409618

mod avg:  0.15483845337299693
mod max:  0.20068479073837478
mod min:  0.062367443624475404

C_S avg:  21.449868219764284
C_S max:  55.22896678510617
C_S min:  5.255752777989293

P_S avg:  0.01546726085362449
P_S max:  0.022222222222222223
P_S min:  0.0106951871657754
Done


In [29]:
def WSpiUpd(nodes,edges,population=20,generation=100,P_se=0.05, P_su = 0.05, P_rs = 0.1, r=1.5):
    
#     common part in both
    
    graph=nx.Graph() 
    graph.add_nodes_from(nodes) #adds nodes
    graph.add_edges_from(edges) #add edges
    Adj = nx.adjacency_matrix(graph) 
    nodes_length = len(graph.nodes())
#     nx.draw(graph, with_labels=True,node_color = "red")
#     plt.show()
    d = {"chrom":[generate_chrom(nodes_length,Adj) for n in range(population)]}
    dframe = pd.DataFrame(data= d)
    dframe["subsets"] = dframe["chrom"].apply(find_subsets)
    dframe["community_score"]=dframe.apply(lambda x: community_score(x["chrom"],x["subsets"],1.5,Adj),axis=1)
    population_count = population
    gen = 0
    
#     velocity and variables of pso
    
    W = 0.8
    c1 = 0.3
    c2 = 0.7
    V = [np.random.randint(-nodes_length/3, nodes_length/3)]*len(dframe)
    dframe["vel"] = V
    
#     take global best and find it

    globalbest = dframe.sort_values("community_score",ascending=False).index[0]
    globalbest = dframe.loc[globalbest]
    
    tempbest = globalbest
    while gen < generation:
       
        for i in range(int(np.floor(population/10))):
#         for i in range(population):
            
            
            
            
            
            r = np.random.rand()
            a = (2-2*gen)/generation
            M = (2*r-1)*a 
            dtemp = dframe.loc[i]
            
#             dtemp = ChangingVelocity(dtemp, globalbest, tempbest, Adj, W, c1, c2, r)
#             dtemp = ShrinkingEncircling(dtemp,P_se, Adj)
#             dtemp = RandomSearchingOperation(dtemp,P_rs,Adj)
            dtemp = SpiralUpdatingOperation(dtemp,P_su,globalbest,Adj)
            
            
#             if r < 0.5:
#                 if abs(M) < 1:
#                     dtemp = ShrinkingEncircling(dtemp,P_se, Adj)
# #                     dtemp = ChangingVelocity(dtemp, globalbest, tempbest, Adj, W, c1, c2, r)
                    
#                 elif abs(M)>=1:
#                     dtemp = RandomSearchingOperation(dtemp,P_rs,Adj)
# #                     dtemp = ChangingVelocity(dtemp, globalbest, tempbest, Adj, W, c1, c2, r)
                    
#             elif r>=0.5:
# #                 dtemp = SpiralUpdatingOperation(dtemp,P_su,globalbest,Adj)
#                 dtemp = ChangingVelocity(dtemp, globalbest, tempbest, Adj, W, c1, c2, r)

                
# #                 dtemp = ChangingVelocity(dtemp, globalbest, tempbest, Adj, W, c1, c2, r)
            
#                       ..............
            if(dtemp['community_score']>dframe.loc[i]['community_score']):
                dframe.at[i,'subsets'] = dtemp['subsets']
                dframe.at[i,'chrom'] = dtemp['chrom']
                dframe.at[i,'community_score'] = dtemp['community_score']
            dframe.at[i, 'vel'] = dtemp['vel']
            
            
            
          
        
        
#         update global
        tempbest = dframe.sort_values("community_score",ascending=False).index[0]
        tempbest = dframe.loc[tempbest]
        
        if globalbest["community_score"]<tempbest["community_score"]:
            globalbest = tempbest
                           
        gen +=1   
            
            
#     end copy
    res = globalbest
#     score = res["community_score"]
    nodes_subsets = res["subsets"]
    res = res['chrom']
    CS, PS = score_calc(res, nodes_subsets, r, Adj)
#     istrue = check_res(res,Adj)
#     print(istrue)
    nodes_list = list(graph.nodes())
    result = []
    for subs in nodes_subsets:
        subset = []
        for n in subs:
            subset.append(nodes_list[n])
        result.append(subset)
    NMI = 0
    clu = globalbest
    clu = clu['chrom']
    clu = np.array(clu)
    for index, target in dframe.iterrows():
        temp = np.array(target['chrom'])
        x = computeNMI(clu,temp)
        NMI += x
    NMI /= len(dframe)
    modularity = nx_comm.modularity(graph, result)
    return result, NMI, modularity, CS, PS
print("WSpiUpd loaded to use")

WSpiUpd loaded to use


In [30]:
NMI_arr_WSpiUpd = []
mod_arr_WSpiUpd = []
bst_arr_WSpiUpd = []
C_S_arr_WSpiUpd = []
P_S_arr_WSpiUpd = []

for i in range(0,20):
    print("WSpiUpd",i+1, end=" ")
    best_community, NMI, modularity,CS, PS = WSpiUpd(nodes,edges)
    NMI_arr_WSpiUpd.append(NMI)
    mod_arr_WSpiUpd.append(modularity)
    bst_arr_WSpiUpd.append(best_community)
    C_S_arr_WSpiUpd.append(CS)
    P_S_arr_WSpiUpd.append(PS)
    

NMI_avg_WSpiUpd = sum(NMI_arr_WSpiUpd)/len(NMI_arr_WSpiUpd)
mod_avg_WSpiUpd = sum(mod_arr_WSpiUpd)/len(mod_arr_WSpiUpd)
C_S_avg_WSpiUpd = sum(C_S_arr_WSpiUpd)/len(C_S_arr_WSpiUpd)
P_S_avg_WSpiUpd = sum(P_S_arr_WSpiUpd)/len(P_S_arr_WSpiUpd)


NMI_bst_WSpiUpd = max(NMI_arr_WSpiUpd)  #best
mod_bst_WSpiUpd = max(mod_arr_WSpiUpd)
C_S_bst_WSpiUpd = max(C_S_arr_WSpiUpd)
P_S_bst_WSpiUpd = max(P_S_arr_WSpiUpd)

NMI_lst_WSpiUpd = min(NMI_arr_WSpiUpd)  #least
mod_lst_WSpiUpd = min(mod_arr_WSpiUpd)
C_S_lst_WSpiUpd = min(C_S_arr_WSpiUpd)
P_S_lst_WSpiUpd = min(P_S_arr_WSpiUpd)




print("WSpiUpd")
print('NMI avg: ', NMI_avg_WSpiUpd)
print('NMI max: ', NMI_bst_WSpiUpd)
print('NMI min: ', NMI_lst_WSpiUpd)
print()
print('mod avg: ', mod_avg_WSpiUpd)
print('mod max: ', mod_bst_WSpiUpd)
print('mod min: ', mod_lst_WSpiUpd)
print()
print('C_S avg: ', C_S_avg_WSpiUpd)
print('C_S max: ', C_S_bst_WSpiUpd)
print('C_S min: ', C_S_lst_WSpiUpd)
print()
print('P_S avg: ', P_S_avg_WSpiUpd)
print('P_S max: ', P_S_bst_WSpiUpd)
print('P_S min: ', P_S_lst_WSpiUpd)
print("Done")

WSpiUpd 1 WSpiUpd 2 WSpiUpd 3 WSpiUpd 4 WSpiUpd 5 WSpiUpd 6 WSpiUpd 7 WSpiUpd 8 WSpiUpd 9 WSpiUpd 10 WSpiUpd 11 WSpiUpd 12 WSpiUpd 13 WSpiUpd 14 WSpiUpd 15 WSpiUpd 16 WSpiUpd 17 WSpiUpd 18 WSpiUpd 19 WSpiUpd 20 WSpiUpd
NMI avg:  0.8761286939565585
NMI max:  0.8834181973631452
NMI min:  0.8596868059472984

mod avg:  0.1546004610472298
mod max:  0.20545205299500602
mod min:  0.06411861796007794

C_S avg:  23.56244723963504
C_S max:  49.316891563519995
C_S min:  7.4996151702835805

P_S avg:  0.018218601993668836
P_S max:  0.027472527472527472
P_S min:  0.013636363636363637
Done


In [31]:
table = ([["", "PSO", "WOA", "GWO", "GWOWOAlinear", "WOAGWOlinear", "GWOWOAMix", "PW1_SE", "PW2_RU", "PW3_SU", "PWVel", "WShrEnc", "WRanUpd", "WSpiUpd"],
         ["NMIAvg",NMI_avg_PSO,NMI_avg_WOA,NMI_avg_GWO,NMI_avg_GWOWOAlinear,NMI_avg_WOAGWOlinear,NMI_avg_GWOWOAMix,NMI_avg_PW1,NMI_avg_PW2,NMI_avg_PW3,NMI_avg_PWVel,NMI_avg_WShrEnc,NMI_avg_WRanUpd,NMI_avg_WSpiUpd],
         ["NMIMax",NMI_bst_PSO,NMI_bst_WOA,NMI_bst_GWO,NMI_bst_GWOWOAlinear,NMI_bst_WOAGWOlinear,NMI_bst_GWOWOAMix,NMI_bst_PW1,NMI_bst_PW2,NMI_bst_PW3,NMI_bst_PWVel,NMI_bst_WShrEnc,NMI_bst_WRanUpd,NMI_bst_WSpiUpd],
         ["NMIMin",NMI_lst_PSO,NMI_lst_WOA,NMI_lst_GWO,NMI_lst_GWOWOAlinear,NMI_lst_WOAGWOlinear,NMI_lst_GWOWOAMix,NMI_lst_PW1,NMI_lst_PW2,NMI_lst_PW3,NMI_lst_PWVel,NMI_lst_WShrEnc,NMI_lst_WRanUpd,NMI_lst_WSpiUpd],
         ["ModAvg",mod_avg_PSO,mod_avg_WOA,mod_avg_GWO,mod_avg_GWOWOAlinear,mod_avg_WOAGWOlinear,mod_avg_GWOWOAMix,mod_avg_PW1,mod_avg_PW2,mod_avg_PW3,mod_avg_PWVel,mod_avg_WShrEnc,mod_avg_WRanUpd,mod_avg_WSpiUpd],
         ["ModMax",mod_bst_PSO,mod_bst_WOA,mod_bst_GWO,mod_bst_GWOWOAlinear,mod_bst_WOAGWOlinear,mod_bst_GWOWOAMix,mod_bst_PW1,mod_bst_PW2,mod_bst_PW3,mod_bst_PWVel,mod_bst_WShrEnc,mod_bst_WRanUpd,mod_bst_WSpiUpd],
         ["ModMin",mod_lst_PSO,mod_lst_WOA,mod_lst_GWO,mod_lst_GWOWOAlinear,mod_lst_WOAGWOlinear,mod_lst_GWOWOAMix,mod_lst_PW1,mod_lst_PW2,mod_lst_PW3,mod_lst_PWVel,mod_lst_WShrEnc,mod_lst_WRanUpd,mod_lst_WSpiUpd],
         ["CSAvg",C_S_avg_PSO,C_S_avg_WOA,C_S_avg_GWO,C_S_avg_GWOWOAlinear,C_S_avg_WOAGWOlinear,C_S_avg_GWOWOAMix,C_S_avg_PW1,C_S_avg_PW2,C_S_avg_PW3,C_S_avg_PWVel,C_S_avg_WShrEnc,C_S_avg_WRanUpd,C_S_avg_WSpiUpd],
         ["CSMax",C_S_bst_PSO,C_S_bst_WOA,C_S_bst_GWO,C_S_bst_GWOWOAlinear,C_S_bst_WOAGWOlinear,C_S_bst_GWOWOAMix,C_S_bst_PW1,C_S_bst_PW2,C_S_bst_PW3,C_S_bst_PWVel,C_S_bst_WShrEnc,C_S_bst_WRanUpd,C_S_bst_WSpiUpd],
         ["CSMin",C_S_lst_PSO,C_S_lst_WOA,C_S_lst_GWO,C_S_lst_GWOWOAlinear,C_S_lst_WOAGWOlinear,C_S_lst_GWOWOAMix,C_S_lst_PW1,C_S_lst_PW2,C_S_lst_PW3,C_S_lst_PWVel,C_S_lst_WShrEnc,C_S_lst_WRanUpd,C_S_lst_WSpiUpd],
         ["PSAvg",P_S_avg_PSO,P_S_avg_WOA,P_S_avg_GWO,P_S_avg_GWOWOAlinear,P_S_avg_WOAGWOlinear,P_S_avg_GWOWOAMix,P_S_avg_PW1,P_S_avg_PW2,P_S_avg_PW3,P_S_avg_PWVel,P_S_avg_WShrEnc,P_S_avg_WRanUpd,P_S_avg_WSpiUpd],
         ["PSMax",P_S_bst_PSO,P_S_bst_WOA,P_S_bst_GWO,P_S_bst_GWOWOAlinear,P_S_bst_WOAGWOlinear,P_S_bst_GWOWOAMix,P_S_bst_PW1,P_S_bst_PW2,P_S_bst_PW3,P_S_bst_PWVel,P_S_bst_WShrEnc,P_S_bst_WRanUpd,P_S_bst_WSpiUpd],
         ["PSMin",P_S_lst_PSO,P_S_lst_WOA,P_S_lst_GWO,P_S_lst_GWOWOAlinear,P_S_lst_WOAGWOlinear,P_S_lst_GWOWOAMix,P_S_lst_PW1,P_S_lst_PW2,P_S_lst_PW3,P_S_lst_PWVel,P_S_lst_WShrEnc,P_S_lst_WRanUpd,P_S_lst_WSpiUpd]])
print(type(table))

<class 'list'>


In [32]:
from openpyxl import Workbook


wb = Workbook() # creates a workbook object.
ws = wb.active # creates a worksheet object.

for row in table:
    ws.append(row) # adds values to cells, each list is a new row.

    
wb.save('Results_C-elegans-frontal.xlsx') # save to excel file.
print("Done")

Done
